In [4]:
import sys
import os
import time
import torch
import datasets
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments
from models.unified import finetune, prefixtuning
from models.unified.prefixtuning import Model

import nltk

# from filelock import FileLock
# with FileLock(".lock") as lock:
#     nltk.download("punkt", quiet=True)
#     nltk.download("stopwords", quiet=True)

2022-12-05 21:23:58.498318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 21:23:58.498346: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
import json
from copy import deepcopy
from collections import Counter, defaultdict
import importlib
import pickle
from typing import List, Dict
from types import SimpleNamespace

from seq2seq_construction import spider
from third_party.spider.preprocess.get_tables import dump_db_json_schema
from third_party.miscs.bridge_content_encoder import get_database_matches

import numpy as np
from tqdm.notebook import tqdm
# import editdistance
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize.treebank import TreebankWordDetokenizer

# from SpeakQL.Allennlp_models.utils.spider import process_sql, evaluation
# from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, \
#     Postprocess_rewrite_seq, Postprocess_rewrite_seq_freeze_POS, Postprocess_rewrite_seq_modify_POS

In [6]:
from language.xsp.data_preprocessing import spider_preprocessing, wikisql_preprocessing, michigan_preprocessing

import sdr_analysis
from sdr_analysis.helpers.general_helpers import db_dict_to_general_fmt
from sdr_analysis.helpers.link_prediction_collector import collect_link_prediction_samples
from sdr_analysis.helpers.single_node_reconstruction_collector import collect_single_node_reconstruction_samples
from sdr_analysis.helpers.base_graph_data_collector import BaseGraphDataCollector, BaseGraphDataCollector_spider, BaseGraphDataCollector_wikisql

from sdra.link_prediction_collectors import LinkPredictionDataCollector_USKG_spider, LinkPredictionDataCollector_USKG_wikisql
from sdra.single_node_reconstruction_collectors import SingleNodeReconstructionDataCollector_USKG_spider, SingleNodeReconstructionDataCollector_USKG_wikisql


In [28]:
# import sdr_analysis.helpers.base_graph_data_collector
# importlib.reload(sdr_analysis.helpers.base_graph_data_collector)
# import sdra.probing_data_collectors
# importlib.reload(sdra.probing_data_collectors)

# import sdra.probing_data_collectors
# importlib.reload(sdra.probing_data_collectors)
# import sdra.link_prediction_collectors
# importlib.reload(sdra.link_prediction_collectors)

# import sdra.single_node_reconstruction_collectors
# importlib.reload(sdra.single_node_reconstruction_collectors)
# from sdra.single_node_reconstruction_collectors import SingleNodeReconstructionDataCollector_USKG_spider, SingleNodeReconstructionDataCollector_USKG_wikisql

# import sdr_analysis.helpers.general_helpers
# importlib.reload(sdr_analysis.helpers.general_helpers)
# from sdr_analysis.helpers.general_helpers import load_pickle_list


In [75]:
# might not work because of importing order... just as a sanity check
for m_name, m in sys.modules.items():
    if m_name.startswith('sdr'):
        print(m_name)
        importlib.reload(m)

sdr_analysis
sdr_analysis.helpers
sdr_analysis.helpers.general_helpers
sdr_analysis.helpers.link_prediction_collector
sdr_analysis.helpers.base_graph_data_collector
sdr_analysis.helpers.single_node_reconstruction_collector
sdra
sdra.link_prediction_collectors
sdra.probing_data_utils
sdra.probing_data_collectors
sdra.single_node_reconstruction_collectors


## Read data - Schema

### Spider

#### Original loading test

In [20]:
schema_cache = dict()

db_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/database'

def format_spider_schema(db_id):
    if db_id not in schema_cache:
        schema_cache[db_id] = dump_db_json_schema(
            db_path + "/" + db_id + "/" + db_id + ".sqlite", db_id)
    schema = schema_cache[db_id]

    return {
        "db_id": db_id,
        "db_path": db_path,
        "db_table_names": schema["table_names_original"],
        "db_column_names": {
            "table_id": [table_id for table_id, column_name in schema["column_names_original"]],
            "column_name": [column_name for table_id, column_name in schema["column_names_original"]]
        },
        "db_column_types": schema["column_types"],
        "db_primary_keys": [{"column_id": column_id} for column_id in schema["primary_keys"]],
        "db_foreign_keys": [
            {"column_id": column_id, "other_column_id": other_column_id}
            for column_id, other_column_id in schema["foreign_keys"]
        ],
    }

In [166]:
fmt_schema = format_spider_schema('world_1')
fmt_schema

{'db_id': 'world_1',
 'db_path': '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/database',
 'db_table_names': ['city', 'sqlite_sequence', 'country', 'countrylanguage'],
 'db_column_names': {'table_id': [-1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   3,
   3,
   3,
   3],
  'column_name': ['*',
   'ID',
   'Name',
   'CountryCode',
   'District',
   'Population',
   'name',
   'seq',
   'Code',
   'Name',
   'Continent',
   'Region',
   'SurfaceArea',
   'IndepYear',
   'Population',
   'LifeExpectancy',
   'GNP',
   'GNPOld',
   'LocalName',
   'GovernmentForm',
   'HeadOfState',
   'Capital',
   'Code2',
   'CountryCode',
   'Language',
   'IsOfficial',
   'Percentage']},
 'db_column_types': ['text',
  'number',
  'text',
  'text',
  'text',
  'number',
  'text',
  'text',
  'text',
  'text',
  'text',
  'text',
  'number',
  'number',
  'number',
  'number',
  'number',
  'number',
  'te

In [22]:
schema_cache['concert_singer']

{'db_id': 'concert_singer',
 'table_names_original': ['stadium', 'singer', 'concert', 'singer_in_concert'],
 'table_names': ['stadium', 'singer', 'concert', 'singer in concert'],
 'column_names_original': [(-1, '*'),
  (0, 'Stadium_ID'),
  (0, 'Location'),
  (0, 'Name'),
  (0, 'Capacity'),
  (0, 'Highest'),
  (0, 'Lowest'),
  (0, 'Average'),
  (1, 'Singer_ID'),
  (1, 'Name'),
  (1, 'Country'),
  (1, 'Song_Name'),
  (1, 'Song_release_year'),
  (1, 'Age'),
  (1, 'Is_male'),
  (2, 'concert_ID'),
  (2, 'concert_Name'),
  (2, 'Theme'),
  (2, 'Stadium_ID'),
  (2, 'Year'),
  (3, 'concert_ID'),
  (3, 'Singer_ID')],
 'column_names': [(-1, '*'),
  (0, 'stadium id'),
  (0, 'location'),
  (0, 'name'),
  (0, 'capacity'),
  (0, 'highest'),
  (0, 'lowest'),
  (0, 'average'),
  (1, 'singer id'),
  (1, 'name'),
  (1, 'country'),
  (1, 'song name'),
  (1, 'song release year'),
  (1, 'age'),
  (1, 'is male'),
  (2, 'concert id'),
  (2, 'concert name'),
  (2, 'theme'),
  (2, 'stadium id'),
  (2, 'year'),


In [23]:
list(enumerate(fmt_schema['db_column_names']['column_name']))

[(0, '*'),
 (1, 'Stadium_ID'),
 (2, 'Location'),
 (3, 'Name'),
 (4, 'Capacity'),
 (5, 'Highest'),
 (6, 'Lowest'),
 (7, 'Average'),
 (8, 'Singer_ID'),
 (9, 'Name'),
 (10, 'Country'),
 (11, 'Song_Name'),
 (12, 'Song_release_year'),
 (13, 'Age'),
 (14, 'Is_male'),
 (15, 'concert_ID'),
 (16, 'concert_Name'),
 (17, 'Theme'),
 (18, 'Stadium_ID'),
 (19, 'Year'),
 (20, 'concert_ID'),
 (21, 'Singer_ID')]

#### New loading from nested dict

In [ ]:
xsp_data_dir = "/Users/mac/Desktop/syt/Deep-Learning/Repos/Google-Research-Language/language/language/xsp/data"

spider_tables_path = os.path.join(xsp_data_dir, 'spider', 'tables.json')

spider_dbs_dict = spider_preprocessing.load_spider_tables(spider_tables_path)

In [5]:
spider_dbs_dict.keys()

dict_keys(['perpetrator', 'college_2', 'flight_company', 'icfp_1', 'body_builder', 'storm_record', 'pilot_record', 'race_track', 'academic', 'department_store', 'music_4', 'insurance_fnol', 'cinema', 'decoration_competition', 'phone_market', 'store_product', 'assets_maintenance', 'student_assessment', 'dog_kennels', 'music_1', 'company_employee', 'farm', 'solvency_ii', 'city_record', 'swimming', 'flight_2', 'election', 'manufactory_1', 'debate', 'network_2', 'local_govt_in_alabama', 'climbing', 'e_learning', 'scientist_1', 'ship_1', 'entertainment_awards', 'allergy_1', 'imdb', 'products_for_hire', 'candidate_poll', 'chinook_1', 'flight_4', 'pets_1', 'dorm_1', 'journal_committee', 'flight_1', 'medicine_enzyme_interaction', 'local_govt_and_lot', 'station_weather', 'shop_membership', 'driving_school', 'concert_singer', 'music_2', 'sports_competition', 'railway', 'inn_1', 'museum_visit', 'browser_web', 'baseball_1', 'architecture', 'csu_1', 'tracking_orders', 'insurance_policies', 'gas_com

In [6]:
spider_dbs_dict['concert_singer']

{'stadium': [{'field name': 'Stadium_ID',
   'is primary key': True,
   'is foreign key': True,
   'type': 'number'},
  {'field name': 'Location',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'Name',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'Capacity',
   'is primary key': False,
   'is foreign key': False,
   'type': 'number'},
  {'field name': 'Highest',
   'is primary key': False,
   'is foreign key': False,
   'type': 'number'},
  {'field name': 'Lowest',
   'is primary key': False,
   'is foreign key': False,
   'type': 'number'},
  {'field name': 'Average',
   'is primary key': False,
   'is foreign key': False,
   'type': 'number'}],
 'singer': [{'field name': 'Singer_ID',
   'is primary key': True,
   'is foreign key': True,
   'type': 'number'},
  {'field name': 'Name',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'Country',
   'is

In [7]:
# def db_dict_to_fmt_schema(db_dict):
#     """
#     Args:
#         db_dict: Dict[table_name, List[column_dict["field name", "is primary key", "is foreign key", "type"]]]
    
#     Output:
#         fmt_schema = (for reference) {
#             "db_table_names": schema["table_names_original"],
#             "db_column_names": {
#                 "table_id": [table_id for table_id, column_name in schema["column_names_original"]],
#                 "column_name": [column_name for table_id, column_name in schema["column_names_original"]]
#             },
#             "db_column_types": schema["column_types"],
#             "db_primary_keys": [{"column_id": column_id} for column_id in schema["primary_keys"]],
#             "db_foreign_keys": [
#                 {"column_id": column_id, "other_column_id": other_column_id}
#                 for column_id, other_column_id in schema["foreign_keys"]
#             ],
#         }
#     """

#     db_table_names = []
#     db_column_names = ['*']  # default values in spider, same below 
#     db_column_table_ids = [-1]
#     db_column_types = ['text']
#     db_primary_keys = []
#     db_foreign_keys = []
    
#     # for a column name, find the primary key idx. If this is a foreign key and not primary key, then found a f-p pair
#     # (assume the f-p pair have the same name)
#     # This is not always true. Example DB: architecture::architect_id, college_1::PROF_NUM
#     col_name2p_key_column_idx = dict()
#     col_name2f_keys_column_idx = defaultdict(list)
    
#     for table_name, table_columns in db_dict.items():
#         table_idx = len(db_table_names)
#         db_table_names.append(table_name)
        
#         for col_dict in table_columns:
#             col_idx = len(db_column_names)
#             db_column_names.append(col_dict["field name"])
#             db_column_table_ids.append(table_idx)
#             db_column_types.append(col_dict["type"])
            
#             # in michigan datasets, "is primary||foreign key" is "y"/"n"; in spider and wikisql, it is true/false
#             if col_dict["is primary key"] in {True, 'y'}:
#                 db_primary_keys.append({"column_id": col_idx})
#                 if col_dict["field name"] in col_name2p_key_column_idx:
#                     ## already exists?? Yes, there could be f-p pairs where both are primary! 
#                     # print(f'Warning: {col_dict["field name"]} already exists')
#                     pass
#                 else:
#                     col_name2p_key_column_idx[col_dict["field name"]] = col_idx
            
#             if col_dict["is foreign key"] in {True, 'y'}:
#                 col_name2f_keys_column_idx[col_dict["field name"]].append(col_idx)
        
#     for col_name in col_name2f_keys_column_idx.keys():
#         f_key_column_ids = col_name2f_keys_column_idx[col_name]
#         if col_name in col_name2p_key_column_idx:
#             p_key_column_idx = col_name2p_key_column_idx[col_name]
#         else:
#             print(f'Warning: {col_name}, no primary key found')
#             continue
            
#         for f_key_column_idx in f_key_column_ids:
#             if f_key_column_idx != p_key_column_idx:
#                 db_foreign_keys.append({"column_id": f_key_column_idx, "other_column_id": p_key_column_idx})
    
#     return {
#         "db_table_names": db_table_names,
#         "db_column_names": {
#             "table_id": db_column_table_ids,
#             "column_name": db_column_names,
#         },
#         "db_column_types": db_column_types,
#         "db_primary_keys": db_primary_keys,
#         "db_foreign_keys": db_foreign_keys
#     }


In [43]:
def general_fmt_dict_to_uskg_schema(general_fmt_dict):
    """
    Args:
        general_fmt_dict (Dict): {
            "db_id": str
            "table_names_original": List[str], original table name (concert_singer)
            "table_names_clean": List[str], clean table names (concert_singer)
            "column_names_original": List[str], original column name (singer_id)
            "column_names_clean": List[str], clean columns names (singer id)
            "column_db_full_names": List[str], name of table::column in DB (may differ from column_names) (singer::singer_id)
            "column_table_ids": List[int], for each column, the corresponding table index
            "column_types": List[str], column types
            "primary_keys": List[int], the columns indices that are primary key
            "foreign_keys": List[[int, int]], the f-p column index pairs (fk_id, pk_id)
            "sqlite_path": str
            "sqlite_conn": sqlite3.Connection
        }
    
    Output:
        uskg_schema = (for reference) {
            "db_table_names": schema["table_names_original"],
            "db_column_names": {
                "table_id": [table_id for table_id, column_name in schema["column_names_original"]],
                "column_name": [column_name for table_id, column_name in schema["column_names_original"]]
            },
            "db_column_types": schema["column_types"],
            "db_primary_keys": [{"column_id": column_id} for column_id in schema["primary_keys"]],
            "db_foreign_keys": [
                {"column_id": column_id, "other_column_id": other_column_id}
                for column_id, other_column_id in schema["foreign_keys"]
            ],
        }
    """

    db_id = general_fmt_dict["db_id"]
    db_table_orig_names = general_fmt_dict["table_names_original"]
    db_table_clean_names = general_fmt_dict["table_names_clean"]
    db_column_orig_names = general_fmt_dict["column_names_original"]
    db_column_clean_names = general_fmt_dict["column_names_clean"]
    col_db_full_names = general_fmt_dict["column_db_full_names"]
    db_column_table_ids = general_fmt_dict["column_table_ids"]
    db_column_types = general_fmt_dict["column_types"]
    db_primary_keys = general_fmt_dict["primary_keys"]
    db_foreign_keys = general_fmt_dict["foreign_keys"]
    sqlite_path = general_fmt_dict["sqlite_path"]
    sqlite_conn = general_fmt_dict["sqlite_conn"]
    
    # USKG specific
    uskg_primary_keys = [{"column_id": col_idx} for col_idx in db_primary_keys]
    uskg_foreign_keys = [{"column_id": fk_idx, "other_column_id": pk_idx} for fk_idx, pk_idx in db_foreign_keys]

    uskg_schema = {
        "db_id": db_id,
        "db_table_names": db_table_orig_names,
        "db_column_names": {
            "table_id": db_column_table_ids,
            "column_name": db_column_orig_names,
        },
        "db_column_types": db_column_types,
        "db_primary_keys": [{"column_id": column_id} for column_id in db_primary_keys],
        "db_foreign_keys": [
            {"column_id": column_id, "other_column_id": other_column_id}
            for column_id, other_column_id in db_foreign_keys
        ],
    }
    
    return uskg_schema


In [44]:
db_id = 'architecture'
db_dict = spider_dbs_dict[db_id]

general_fmt_dict = db_dict_to_general_fmt(db_dict, db_id,
                                          sqlite_path=f"/Users/mac/Desktop/syt/Deep-Learning/Repos/Google-Research-Language/language/language/xsp/data/spider/database/{db_id}/{db_id}.sqlite",
                                          rigorous_foreign_key=True)

In [45]:
general_fmt_dict

{'db_id': 'architecture',
 'table_names_original': ['architect', 'bridge', 'mill'],
 'table_names_clean': ['architect', 'bridge', 'mill'],
 'column_names_original': ['*',
  'id',
  'name',
  'nationality',
  'gender',
  'architect_id',
  'id',
  'name',
  'location',
  'length_meters',
  'length_feet',
  'architect_id',
  'id',
  'location',
  'name',
  'type',
  'built_year',
  'notes'],
 'column_names_clean': ['*',
  'id',
  'name',
  'nationality',
  'gender',
  'architect id',
  'id',
  'name',
  'location',
  'length meters',
  'length feet',
  'architect id',
  'id',
  'location',
  'name',
  'type',
  'built year',
  'notes'],
 'column_db_full_names': ['NONE::*',
  'architect::id',
  'architect::name',
  'architect::nationality',
  'architect::gender',
  'bridge::architect_id',
  'bridge::id',
  'bridge::name',
  'bridge::location',
  'bridge::length_meters',
  'bridge::length_feet',
  'mill::architect_id',
  'mill::id',
  'mill::location',
  'mill::name',
  'mill::type',
  'mil

In [46]:
fmt_schema_2 = general_fmt_dict_to_uskg_schema(general_fmt_dict)

In [47]:
fmt_schema_2

{'db_id': 'architecture',
 'db_table_names': ['architect', 'bridge', 'mill'],
 'db_column_names': {'table_id': [-1,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   2,
   2,
   2,
   2,
   2,
   2,
   2],
  'column_name': ['*',
   'id',
   'name',
   'nationality',
   'gender',
   'architect_id',
   'id',
   'name',
   'location',
   'length_meters',
   'length_feet',
   'architect_id',
   'id',
   'location',
   'name',
   'type',
   'built_year',
   'notes']},
 'db_column_types': ['text',
  'text',
  'text',
  'text',
  'text',
  'number',
  'number',
  'text',
  'text',
  'number',
  'number',
  'number',
  'number',
  'text',
  'text',
  'text',
  'number',
  'text'],
 'db_primary_keys': [{'column_id': 1}, {'column_id': 6}, {'column_id': 12}],
 'db_foreign_keys': [{'column_id': 5, 'other_column_id': 1},
  {'column_id': 11, 'other_column_id': 1}]}

In [ ]:
fmt_schema = format_spider_schema(db_id)
# del fmt_schema['db_id']
del fmt_schema['db_path']

fmt_schema == fmt_schema_2

In [ ]:
sorted(fmt_schema["db_foreign_keys"], key=lambda d: d['column_id']), sorted(fmt_schema_2["db_foreign_keys"], key=lambda d: d['column_id'])

In [ ]:
list(enumerate(fmt_schema['db_column_names']['column_name']))

### WikiSQL
- Should be able to reuse db_dict_to_fmt_schema()
- KEEP these cells as concrete examples for Wikisql db_dict and general_fmt_dict!

In [94]:
xsp_data_dir = "/home/yshao/Projects/language/language/xsp/data"

In [95]:
wikisql_tables_path = os.path.join(xsp_data_dir, 'wikisql', 'dev.tables.jsonl')

wikisql_dbs_dict = wikisql_preprocessing.load_wikisql_tables(wikisql_tables_path)

In [106]:
# Checking: No primary keys and foreign keys in wikisql

for db_id, db_dict in wikisql_dbs_dict.items():
    col_list = list(db_dict.values())[0]
    for col_dict in col_list:
        if col_dict['is primary key'] or col_dict['is foreign key']:
            print(db_dict)
            break

In [96]:
wikisql_dbs_dict['1-29690363-3'].keys(), wikisql_dbs_dict['1-29690363-3']['RACE_RESULTS']

(dict_keys(['RACE_RESULTS']),
 [{'field name': 'RD',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'RACE',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'POLE_POSITION',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'FASTEST_LAP',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'MOST_LAPS_LED',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'WINNING_DRIVER',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'WINNING_TEAM',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'}])

In [98]:
# db_dict_to_fmt_schema(wikisql_dbs_dict['1-29690363-3'])
db_dict_to_general_fmt(wikisql_dbs_dict['1-29690363-3'], '1-29690363-3', rigorous_foreign_key=False)

{'db_id': '1-29690363-3',
 'table_names_original': ['RACE_RESULTS'],
 'table_names_clean': ['race results'],
 'column_names_original': ['*',
  'RD',
  'RACE',
  'POLE_POSITION',
  'FASTEST_LAP',
  'MOST_LAPS_LED',
  'WINNING_DRIVER',
  'WINNING_TEAM'],
 'column_names_clean': ['*',
  'rd',
  'race',
  'pole position',
  'fastest lap',
  'most laps led',
  'winning driver',
  'winning team'],
 'column_db_full_names': ['NONE::*'],
 'column_table_ids': [-1, 0, 0, 0, 0, 0, 0, 0],
 'column_types': ['text',
  'text',
  'text',
  'text',
  'text',
  'text',
  'text',
  'text'],
 'primary_keys': [],
 'foreign_keys': [],
 'sqlite_path': None,
 'sqlite_conn': None}

### Michigan
- Should be able to reuse db_dict_to_fmt_schema()

In [106]:
atis_schema_path = os.path.join(xsp_data_dir, 'atis', 'atis_schema.csv')

atis_db_dict = michigan_preprocessing.read_schema(atis_schema_path)

In [ ]:
atis_db_dict

In [ ]:
db_dict_to_fmt_schema(atis_db_dict)

## Get USKG encoding

In [5]:
# TODO:
#     - set up inference (for sanity check)
#     - look into source (of transformers Bert.generate) to find ways to get the encoding

### Check USKG inference

In [8]:
def play_pred(txt, model, tokenizer):
    tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
    pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
    )
    return pred

In [9]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take spider for example
            '--cfg', 'Salesforce/T5_large_prefix_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

In [10]:
model_path = 'hkunlp/from_all_T5_large_prefix_spider_with_cell_value2'
# model_path = 'hkunlp/from_all_T5_large_prefix_spider_with_cell_value2'
# model_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-asr_mixed/checkpoint-79500/'
# model_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-rewritten_mixed/checkpoint-56500/'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# for reconstruction
tokenizer_fast = AutoTokenizer.from_pretrained('t5-large', use_fast=True)

model = Model(args)
model.load(model_path)

prefix-tuning sequence length is 10.


In [11]:
# tokenizer_large_fast = AutoTokenizer.from_pretrained('t5-large', use_fast=True)

In [12]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "what is the minimum, average, and maximum age of all singers from France?"

play_pred("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

['select min(age), avg(age), max(age) from singer where country = "France"']

In [13]:
play_pred("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer_fast)

['select min(age), avg(age), max(age) from singer where country = "France"']

### Check USKG inference on dataset samples

In [346]:
xsp_data_dir = "/Users/mac/Desktop/syt/Deep-Learning/Repos/Google-Research-Language/language/language/xsp/data"

#### Spider

In [48]:
orig_dataset_path = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/spider/dev+ratsql_graph.json"
orig_tables_path = "/Users/mac/Desktop/syt/Deep-Learning/Repos/Google-Research-Language/language/language/xsp/data/spider/tables.json"

In [49]:
spider_dbs_dict = spider_preprocessing.load_spider_tables(orig_tables_path)
len(spider_dbs_dict)

166

In [50]:
uskg_schemas_dict = dict()
for db_id, db_dict in spider_dbs_dict.items():
    general_fmt_dict = db_dict_to_general_fmt(db_dict, db_id,
                                              sqlite_path=os.path.join(xsp_data_dir, f"spider/database/{db_id}/{db_id}.sqlite"),
                                              rigorous_foreign_key=True)
    
    uskg_schema = general_fmt_dict_to_uskg_schema(general_fmt_dict)
    
    uskg_schemas_dict[db_id] = uskg_schema
    
len(uskg_schemas_dict)

166

In [124]:
with open(orig_dataset_path, 'r') as f:
    orig_dataset = json.load(f)

for d in orig_dataset:
    d['rat_sql_graph']['relations'] = json.loads(d['rat_sql_graph']['relations'])

len(orig_dataset)

1034

In [125]:
orig_dataset[0].keys()

dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'rat_sql_graph'])

In [126]:
SPIDER_DB_PATH = os.path.join(xsp_data_dir, "spider/database")

def uskg_sample_to_struct_input(uskg_sample):
    db_id = uskg_sample["db_id"]
    uskg_schema = uskg_schemas_dict[db_id]
    
    return spider.serialize_schema(
        question=uskg_sample["question"],
        db_path=SPIDER_DB_PATH,
        db_id=db_id,
        db_column_names=uskg_schema["db_column_names"],
        db_table_names=uskg_schema["db_table_names"],
        schema_serialization_type="peteshaw",
        schema_serialization_randomized=False,
        schema_serialization_with_db_id=True,
        schema_serialization_with_db_content=True,
        normalize_query=True,
    )

In [128]:
idx = 777
sample = orig_dataset[idx]
sample.keys(), sample['rat_sql_graph'].keys()

(dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'rat_sql_graph']),
 dict_keys(['nodes', 'q_nodes_orig', 'relations']))

In [129]:
text_in = sample['question']
struct_in = uskg_sample_to_struct_input(sample)
text_in, struct_in

('What are the Asian countries which have a population larger than that of any country in Africa?',
 ' | world_1 | city : id , name , countrycode , district , population | sqlite_sequence : name , seq | country : code , name , continent ( Africa , Asia ) , region , surfacearea , indepyear , population , lifeexpectancy , gnp , gnpold , localname , governmentform , headofstate , capital , code2 | countrylanguage : countrycode , language , isofficial , percentage')

In [ ]:
play_pred("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer_fast)

#### Wikisql

In [347]:
orig_dataset_path = "/Users/mac/Desktop/syt/Deep-Learning/Repos/Google-Research-Language/language/language/xsp/data/wikisql/dev.jsonl"
orig_tables_path = "/Users/mac/Desktop/syt/Deep-Learning/Repos/Google-Research-Language/language/language/xsp/data/wikisql/dev.tables.jsonl"


In [391]:
# load dataset 
with open(orig_dataset_path, 'r') as f:
    wikisql_dataset = [json.loads(l) for l in f]
len(wikisql_dataset)

8421

In [393]:
sample = wikisql_dataset[0]
sample

{'phase': 1,
 'table_id': '1-10015132-11',
 'question': 'What position does the player who played for butler cc (ks) play?',
 'sql': {'sel': 3, 'conds': [[5, 0, 'Butler CC (KS)']], 'agg': 0}}

In [394]:
# load dbs 
wikisql_dbs_dict = wikisql_preprocessing.load_wikisql_tables(orig_tables_path)
len(wikisql_dbs_dict)

2716

In [395]:
list(wikisql_dbs_dict.keys())[::300]

['1-10015132-11',
 '1-19422702-1',
 '1-27756314-8',
 '2-10932739-3',
 '2-11999396-1',
 '2-13845847-2',
 '2-15442855-1',
 '2-16977283-2',
 '2-17822-4',
 '2-18964684-2']

In [396]:
wikisql_dbs_dict['1-10015132-11']

{'L': [{'field name': 'PLAYER',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'NO',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'NATIONALITY',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'POSITION',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'YEARS_IN_TORONTO',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'},
  {'field name': 'SCHOOL_OR_CLUB_TEAM',
   'is primary key': False,
   'is foreign key': False,
   'type': 'text'}]}

In [397]:
# for wikisql, get the sqlite_conn first 
db_id = sample['table_id']
db_dict = wikisql_dbs_dict[db_id]
general_fmt_dict = db_dict_to_general_fmt(db_dict, db_id,
                                          sqlite_path=os.path.join(xsp_data_dir, "wikisql/dev.db"),
                                          rigorous_foreign_key=False)

wikisql_sqlite_conn = general_fmt_dict['sqlite_conn']
type(wikisql_sqlite_conn)

sqlite3.Connection

In [398]:
general_fmt_dict

{'db_id': '1-10015132-11',
 'table_names_original': ['L'],
 'table_names_clean': ['l'],
 'column_names_original': ['*',
  'PLAYER',
  'NO',
  'NATIONALITY',
  'POSITION',
  'YEARS_IN_TORONTO',
  'SCHOOL_OR_CLUB_TEAM'],
 'column_names_clean': ['*',
  'player',
  'no',
  'nationality',
  'position',
  'years in toronto',
  'school or club team'],
 'column_db_full_names': ['NONE::*'],
 'column_table_ids': [-1, 0, 0, 0, 0, 0, 0],
 'column_types': ['text', 'text', 'text', 'text', 'text', 'text', 'text'],
 'primary_keys': [],
 'foreign_keys': [],
 'sqlite_path': '/Users/mac/Desktop/syt/Deep-Learning/Repos/Google-Research-Language/language/language/xsp/data/wikisql/dev.db',
 'sqlite_conn': <sqlite3.Connection at 0x7fa6cf67d2d0>}

In [399]:
uskg_schema = general_fmt_dict_to_uskg_schema(general_fmt_dict)

In [400]:
uskg_schema

{'db_id': '1-10015132-11',
 'db_table_names': ['L'],
 'db_column_names': {'table_id': [-1, 0, 0, 0, 0, 0, 0],
  'column_name': ['*',
   'PLAYER',
   'NO',
   'NATIONALITY',
   'POSITION',
   'YEARS_IN_TORONTO',
   'SCHOOL_OR_CLUB_TEAM']},
 'db_column_types': ['text', 'text', 'text', 'text', 'text', 'text', 'text'],
 'db_primary_keys': [],
 'db_foreign_keys': []}

In [380]:
# look into wikisql db 
def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

rows = wikisql_sqlite_conn.execute("PRAGMA table_info({})".format(sql_identifier('table_1_10015132_11')))

In [381]:
for r in rows.fetchall():
    print(dict(r))

{'cid': 0, 'name': 'col0', 'type': 'text', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 1, 'name': 'col1', 'type': 'text', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 2, 'name': 'col2', 'type': 'text', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 3, 'name': 'col3', 'type': 'text', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 4, 'name': 'col4', 'type': 'text', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 5, 'name': 'col5', 'type': 'text', 'notnull': 0, 'dflt_value': None, 'pk': 0}


In [ ]:
# what are the correct table names?? - "table_xxx_xxx" (when db_id = "xxx-xxx")
rows = wikisql_sqlite_conn.execute("SELECT name FROM sqlite_master WHERE type='table'")

In [ ]:
for r in rows.fetchmany(10):
    print(dict(r))

In [ ]:
uskg_schemas_dict = dict()
for db_id, db_dict in wikisql_dbs_dict.items():
    general_fmt_dict = db_dict_to_general_fmt(db_dict, db_id,
                                              sqlite_conn=wikisql_sqlite_conn,
                                              rigorous_foreign_key=False)
    
    uskg_schema = general_fmt_dict_to_uskg_schema(general_fmt_dict)
    
    uskg_schemas_dict[db_id] = uskg_schema
    
len(uskg_schemas_dict)

In [412]:
## schema serialization for wikisql 

def _wikisql_db_id_to_table_name(db_id):
    return '_'.join(['table'] + db_id.split('-'))

def serialize_schema_wikisql(
        question: str,
        db_path: str,
        db_id: str,
        db_column_names: Dict[str, str],
        db_table_names: List[str],
        schema_serialization_type: str = "peteshaw",
        schema_serialization_randomized: bool = False,
        schema_serialization_with_db_id: bool = True,
        schema_serialization_with_db_content: bool = False,
        normalize_query: bool = True,
) -> str:
    if schema_serialization_type == "verbose":
        db_id_str = "Database: {db_id}. "
        table_sep = ". "
        table_str = "Table: {table}. Columns: {columns}"
        column_sep = ", "
        column_str_with_values = "{column} ({values})"
        column_str_without_values = "{column}"
        value_sep = ", "
    elif schema_serialization_type == "peteshaw":
        # see https://github.com/google-research/language/blob/master/language/nqg/tasks/spider/append_schema.py#L42
        db_id_str = " | {db_id}"
        table_sep = ""
        table_str = " | {table} : {columns}"
        column_sep = " , "
        column_str_with_values = "{column} ( {values} )"
        column_str_without_values = "{column}"
        value_sep = " , "
    else:
        raise NotImplementedError

    def get_column_str(table_name: str, column_name: str, sqlite_table_name: str, sqlite_column_name: str) -> str:
        """
        Args:
            table_name: the original table name in wikisql dataset (natural langauge)
            column_name: the name is wikisql dataset (natural langauge)
            sqlite_table_name: the table name in .db (table_xxx_xxx)
            sqlite_column_name: the column name in .db (col0, col1, ...)
        """
        column_name_str = column_name.lower() if normalize_query else column_name
        if schema_serialization_with_db_content:
            matches = get_database_matches(
                question=question,
                table_name=sqlite_table_name,
                column_name=sqlite_column_name,
                # db_path=(db_path + "/" + db_id + "/" + db_id + ".sqlite"),
                db_path=db_path,
            )
            if matches:
                return column_str_with_values.format(
                    column=column_name_str, values=value_sep.join(matches)
                )
            else:
                return column_str_without_values.format(column=column_name_str)
        else:
            return column_str_without_values.format(column=column_name_str)

#     tables = [
#         table_str.format(
#             table=table_name.lower() if normalize_query else table_name,
#             columns=column_sep.join(
#                 map(
#                     lambda y: get_column_str(table_name=table_name,
#                                              column_name=y[1],
#                                              sqlite_table_name=_wikisql_db_id_to_table_name(db_id),
#                                              sqlite_column_name=f'col{y[2]}'),
#                     filter(
#                         lambda y: y[0] == table_id,
#                         zip(
#                             db_column_names["table_id"],
#                             db_column_names["column_name"],
#                             range(len(db_column_names["table_id"])),  # incorrect, should be in-table index! 
#                         ),
#                     ),
#                 )
#             ),
#         )
#         for table_id, table_name in enumerate(db_table_names)
#     ]

    tables = []
    for table_id, table_name in enumerate(db_table_names):
        table_col_idx = 0
        column_str_list = []
        for col_table_id, col_name in zip(db_column_names["table_id"], db_column_names["column_name"]):
            if col_table_id != table_id:
                continue
            column_str_list.append(get_column_str(table_name=table_name,
                                                  column_name=col_name,
                                                  sqlite_table_name=_wikisql_db_id_to_table_name(db_id),
                                                  sqlite_column_name=f'col{table_col_idx}'))
            table_col_idx += 1
            
        tables.append(table_str.format(
            table=table_name.lower() if normalize_query else table_name,
            columns=column_sep.join(column_str_list),
        ))

    if schema_serialization_randomized:
        random.shuffle(tables)
    if schema_serialization_with_db_id:
        serialized_schema = db_id_str.format(db_id=db_id) + table_sep.join(tables)
    else:
        serialized_schema = table_sep.join(tables)
    return serialized_schema



In [413]:
sample, uskg_schema

({'phase': 1,
  'table_id': '1-10015132-11',
  'question': 'What position does the player who played for butler cc (ks) play?',
  'sql': {'sel': 3, 'conds': [[5, 0, 'Butler CC (KS)']], 'agg': 0}},
 {'db_id': '1-10015132-11',
  'db_table_names': ['L'],
  'db_column_names': {'table_id': [-1, 0, 0, 0, 0, 0, 0],
   'column_name': ['*',
    'PLAYER',
    'NO',
    'NATIONALITY',
    'POSITION',
    'YEARS_IN_TORONTO',
    'SCHOOL_OR_CLUB_TEAM']},
  'db_column_types': ['text', 'text', 'text', 'text', 'text', 'text', 'text'],
  'db_primary_keys': [],
  'db_foreign_keys': []})

In [414]:
db_id = sample["table_id"]

struct_in = serialize_schema_wikisql(
    question=sample["question"],
    db_path=os.path.join(xsp_data_dir, "wikisql/dev.db"),
    db_id=db_id,
    db_column_names=uskg_schema["db_column_names"],
    db_table_names=uskg_schema["db_table_names"],
    schema_serialization_type="peteshaw",
    schema_serialization_randomized=False,
    schema_serialization_with_db_id=True,
    schema_serialization_with_db_content=True,
    normalize_query=True,
)

In [415]:
struct_in

' | 1-10015132-11 | l : player , no , nationality , position , years_in_toronto , school_or_club_team ( butler cc (ks) )'

### Tokenized pieces-nodes mapping

In [130]:
# TODO (done):
#     find mapping between tokenized pieces to ratsql nodes
#         ratsql nodes to sentence char ids (done)
#         sentence char ids to tokenized pieces (done)
#     pool the encodings of pieces in a node (done)

In [165]:
class StructCharRangesCollector:
    def __init__(self):
        self.initialize()

    def initialize(self):
        self.db_id2char_ranges = dict()
        self.table2char_ranges = dict()
        self.column2char_ranges = dict()
        
        # Due to rat-sql stemming tokens, rat-sql nodes and uskg text may mismatch,
        # so we save a list and use the order instead of name for indexing 
        self.db_id_char_ranges_list = []
        self.column_char_ranges_list = []
        self.table_char_ranges_list = []

        self.bar_cnt = 0
        self.curr_table = None
        self.curr_node_type = None   # [None, 'db_id', 'table', 'column']
        self.curr_node_toks = []
        self.curr_node_char_start = None
        self.curr_node_char_end = None
        self.open_bracket = False
        
    def _register_curr_node(self):
        curr_node_name = ' '.join(self.curr_node_toks)
        curr_range = (self.curr_node_char_start, self.curr_node_char_end)
        
        if self.curr_node_type == 'db_id':
            self.db_id2char_ranges[curr_node_name] = curr_range
            self.db_id_char_ranges_list.append(curr_range)   
        elif self.curr_node_type == 'table':
            self.table2char_ranges[curr_node_name] = curr_range
            self.table_char_ranges_list.append(curr_range)
            self.curr_table = curr_node_name
        elif self.curr_node_type == 'column':
            self.column2char_ranges[(self.curr_table, curr_node_name)] = curr_range
            self.column_char_ranges_list.append(curr_range)
        else:
            raise ValueError(curr_node_type)

        self.curr_node_toks = []
        self.curr_node_char_start = None
        self.curr_node_char_end = None
    
    def collect(self, struct_in, tokenized_txt, _n_words_before_struct):
#         struct_words = struct_in.strip().split(' ')
        struct_words = struct_in.strip().split()
        
        for sw_id, sw in enumerate(struct_words):
            char_range = tokenized_txt.word_to_chars(sw_id + _n_words_before_struct)

            # print(sw_id, char_range, sw, self.curr_node_type, self.open_bracket)

            if sw == '(':
                self.open_bracket = True
                continue

            if sw == ')':
                self.open_bracket = False
                self.curr_node_char_end = char_range[1]
                continue

            if self.open_bracket:
                # in the list of cells, do not add tokens here to name 
                continue

            if sw == '|':
                if self.curr_node_type is not None:
                    self._register_curr_node()
                self.bar_cnt += 1
                if self.bar_cnt == 1:
                    self.curr_node_type = 'db_id'
                if self.bar_cnt > 1:
                    self.curr_node_type = 'table'
                continue

            if sw == ':':
                assert self.curr_node_type == 'table'
                self._register_curr_node()
                self.curr_node_type = 'column'
                continue

            if sw == ',':
                assert self.curr_node_type == 'column'
                self._register_curr_node()
                self.curr_node_type = 'column'
                continue

            self.curr_node_toks.append(sw)
            if self.curr_node_char_start is None:
                self.curr_node_char_start = char_range[0]
            self.curr_node_char_end = char_range[1]

        self._register_curr_node()


In [166]:
## Combine experiment codes 
def collect_node_char_ranges(sample, tokenizer=None, tokenizer_args=None, txt=None, tokenized_txt=None, debug=False):
    text_in = sample['question']
    struct_in = uskg_sample_to_struct_input(sample)
    _splitter = "; structed knowledge: "
    
    if tokenizer_args is None:
        tokenizer_args = dict()
        
    if txt is None:
        txt = "{}{}{}".format(text_in, _splitter, struct_in)
    
    if tokenized_txt is None:
        # tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
        tokenized_txt = tokenizer([txt], **tokenizer_args)
        ## possible problem: exceeding max length!
    
    ratsql_graph_nodes = sample['rat_sql_graph']['nodes']
#     question_toks = sample['question_toks']
    question_toks = sample['rat_sql_graph']['q_nodes_orig']

    _q_nodes = []  # [stem token (node name)]
    q_nodes = []  # [(stem token (node name), orig question token)]
    c_nodes = []  # [(orig table name, orig column name)]
    t_nodes = []  # [orig table name]

    for n in ratsql_graph_nodes:
        if n.startswith('<C>'):
            _n = n[3:]
            _t, _c = _n.split('::')
            c_nodes.append((_t, _c))
        elif n.startswith('<T>'):
            _n = n[3:]
            t_nodes.append(_n)
        else:
            _q_nodes.append(n)

    assert len(_q_nodes) == len(question_toks), (_q_nodes, question_toks)
    q_nodes = list(zip(_q_nodes, question_toks))
    
    # Collection char ranges 
    q_node_chars = []   # [(st, ed)]; same below
    c_node_chars = []
    t_node_chars = []
    
    # Text part
    # Assumption: the mismatch between whitespace words (text_words) and question words only come from trailing puncts
    # Currently the code can handle combining question toks into whitespace words
#     text_words = text_in.strip().split(' ') + ['<SENTINAL>']
    text_words = text_in.lower().strip().split() + ['<SENTINAL>']
    text_word_char_ranges = [tokenized_txt.word_to_chars(i) for i in range(len(text_words) - 1)] + [(None, None)]  # -1 to remove the sentinal 

    curr_tw_idx = 0
    curr_tw = text_words[0]
    curr_tw_char_range = text_word_char_ranges[0]
    curr_char_ptr = 0
    for stem_tok, orig_tok in q_nodes:
        if curr_tw == orig_tok:
            # finishing current word 
            q_node_chars.append((curr_char_ptr, curr_char_ptr + len(orig_tok)))   # curr pos to curr pos + len 
            curr_tw_idx += 1
            curr_tw = text_words[curr_tw_idx]
            curr_tw_char_range = text_word_char_ranges[curr_tw_idx]
            curr_char_ptr = curr_tw_char_range[0]
        else:
            # not finishing current word 
            assert curr_tw.startswith(orig_tok), (curr_tw, orig_tok)
            q_node_chars.append((curr_char_ptr, curr_char_ptr + len(orig_tok)))   # curr pos to curr pos + len 
            curr_char_ptr += len(orig_tok)     # move ptr forward by len 
            curr_tw = curr_tw[len(orig_tok):]  # get the remaining chars in the word 

    assert [txt[st:ed].lower() for st, ed in q_node_chars] == question_toks, ([txt[st:ed] for st, ed in q_node_chars], question_toks)
    
    # Struct part 
    _str_before_struct = text_in + _splitter
    _n_words_before_struct = len(_str_before_struct.strip().split())

    struct_ranges_collector = StructCharRangesCollector()
    struct_ranges_collector.collect(struct_in, tokenized_txt, _n_words_before_struct)
    
    # Due to rat-sql stemming tokens, rat-sql nodes and uskg text may mismatch
#     for c_node in c_nodes:
#         if c_node == ('NONE', '*'):
#             # the special column in spider, using db_id 
#             c_node_chars.append(list(struct_ranges_collector.db_id2char_ranges.values())[0])   # assuming only 1 db_id, which should be true...
#         else:
#             c_node_chars.append(struct_ranges_collector.column2char_ranges[c_node])

#     for t_node in t_nodes:
#         t_node_chars.append(struct_ranges_collector.table2char_ranges[t_node])

    c_node_chars.extend(struct_ranges_collector.db_id_char_ranges_list + struct_ranges_collector.column_char_ranges_list)
    t_node_chars.extend(struct_ranges_collector.table_char_ranges_list)

    ## Check all 
    if debug:
        for q_node, (st, ed) in zip(q_nodes, q_node_chars):
            print(q_node, txt[st:ed])
        print()
        for t_node, (st, ed) in zip(t_nodes, t_node_chars):
            print(t_node, txt[st:ed])
        print()
        for c_node, (st, ed) in zip(c_nodes, c_node_chars):
            print(c_node, txt[st:ed])
        
    return {
        "q_node_chars": q_node_chars,
        "c_node_chars": c_node_chars,
        "t_node_chars": t_node_chars,
    }
    

In [167]:
idx = 209
sample = orig_dataset[idx]

text_in = sample['question']
struct_in = uskg_sample_to_struct_input(sample)

txt = "{}; structed knowledge: {}".format(text_in, struct_in)

tokenized_txt = tokenizer_fast([txt], max_length=1024, padding="max_length", truncation=True)

In [168]:
# tokenizer_args = {
#     "max_length": 1024,
#     "padding": "max_length",
#     "truncation": True
# }

# char_ranges_dict = collect_node_char_ranges(sample, tokenizer=tokenizer_fast, tokenizer_args=tokenizer_args, debug=True)
char_ranges_dict = collect_node_char_ranges(sample, txt=txt, tokenized_txt=tokenized_txt, debug=True)

('how', 'how') How
('many', 'many') many
('flight', 'flights') flights
('arrive', 'arriving') arriving
('in', 'in') in
('aberdeen', 'aberdeen') Aberdeen
('city', 'city') city
('?', '?') ?

airline airlines
airport airports
flight flights

('NONE', '*') flight_2
('airline', 'uid') uid
('airline', 'airline') airline
('airline', 'abbreviation') abbreviation
('airline', 'country') country
('airport', 'city') city ( Aberdeen  )
('airport', 'airportcode') airportcode
('airport', 'airportname') airportname
('airport', 'country') country
('airport', 'countryabbrev') countryabbrev
('flight', 'airline') airline
('flight', 'flightno') flightno
('flight', 'sourceairport') sourceairport
('flight', 'destairport') destairport


In [164]:
txt

'How many flights arriving in Aberdeen city?; structed knowledge:  | flight_2 | airlines : uid , airline , abbreviation , country | airports : city ( Aberdeen  ) , airportcode , airportname , country , countryabbrev | flights : airline , flightno , sourceairport , destairport'

In [169]:
char_ranges_dict

{'q_node_chars': [(0, 3),
  (4, 8),
  (9, 16),
  (17, 25),
  (26, 28),
  (29, 37),
  (38, 42),
  (42, 43)],
 'c_node_chars': [(68, 76),
  (90, 93),
  (96, 103),
  (106, 118),
  (121, 128),
  (142, 160),
  (163, 174),
  (177, 188),
  (191, 198),
  (201, 214),
  (227, 234),
  (237, 245),
  (248, 261),
  (264, 275)],
 't_node_chars': [(79, 87), (131, 139), (217, 224)]}

In [174]:
tokenized_txt.char_to_token(79)

20

In [175]:
tokenized_txt.tokens()[20]

'▁airlines'

In [196]:
sum(tokenized_txt.data['attention_mask'][0])

87

### Get encoding

In [191]:
def get_USKG_node_encodings(sample, model, tokenizer, tokenizer_args=None, pooling_func=None, debug=False):
    """
    Args:
        pooling_func (Callable): np.array(n_pieces, dim) ==> np.array(dim,); default is np.mean
    """
    text_in = sample['question']
    struct_in = uskg_sample_to_struct_input(sample)
    
    _splitter = "; structed knowledge: "
    txt = "{}{}{}".format(text_in, _splitter, struct_in)

    if tokenizer_args is None:
        tokenizer_args = {
            "max_length": 1024,
            "padding": "max_length",
            "truncation": True
        }
    if pooling_func is None:
        pooling_func = lambda l: np.mean(l, axis=0)
        
    tokenized_txt = tokenizer([txt], **tokenizer_args)
    
    # Get encoding tensor 
    with torch.no_grad():
        past_prompt = model.get_prompt(
            bsz=1,              # bsz = input_ids.shape[0]
            sample_size=1,      # sample_size=kwargs['num_beams']
            description=None,   
            knowledge=None,     
        )
        encoder_outputs = model.pretrain_model.encoder(
            input_ids=torch.LongTensor(tokenized_txt.data['input_ids']),
            attention_mask=torch.LongTensor(tokenized_txt.data['attention_mask']),
            past_prompt=past_prompt,
        )
    encoder_output_hidden_states = encoder_outputs.last_hidden_state.detach().squeeze(0).cpu().numpy()
    if debug:
        print('encoder_output_hidden_states:', encoder_output_hidden_states.shape)
    
    # Get node-pieces mapping via char ranges 
    char_ranges_dict = collect_node_char_ranges(sample, txt=txt, tokenized_txt=tokenized_txt)
    node_char_ranges = char_ranges_dict['q_node_chars'] + char_ranges_dict['c_node_chars'] + char_ranges_dict['t_node_chars']

    # some chars can be mapped to multiple tokens (e.g. 'i' => '▁', 'i' )
    char_to_tokens_dict = defaultdict(list)
    for token_idx, tok in enumerate(tokenized_txt.tokens()):
        if tok == '</s>':
            break
        char_span = tokenized_txt.token_to_chars(token_idx)
        for char_idx in range(char_span[0], char_span[1]):
            char_to_tokens_dict[char_idx].append(token_idx)
    
    node_pieces_ranges = []
    for st, ed in node_char_ranges:
        piece_ids = []
        for char_idx in range(st, ed):
            _piece_ids = char_to_tokens_dict[char_idx]
            piece_ids.extend(_piece_ids)

        piece_st = piece_ids[0]
        piece_ed = piece_ids[-1] + 1
        # the collected piece_ids should be continuous 
        # ^ not true... some chars can be mapped to multiple tokens (started by ▁ )
        # re-collect a char-to-token
        assert set(range(piece_st, piece_ed)) == set(piece_ids), piece_ids

        node_pieces_ranges.append((piece_st, piece_ed))
    
    if debug:
        print('node_pieces_ranges:', node_pieces_ranges)
    
    # Pool the encodings per node 
    node_encodings = []
    for piece_st, piece_ed in node_pieces_ranges:
        enc_vecs = encoder_output_hidden_states[piece_st : piece_ed]
        enc_pooled = pooling_func(enc_vecs)
        node_encodings.append(enc_pooled)
    
    return node_encodings

In [177]:
idx = 209
sample = orig_dataset[idx]

text_in = sample['question']
struct_in = uskg_sample_to_struct_input(sample)

txt = "{}; structed knowledge: {}".format(text_in, struct_in)

tokenized_txt = tokenizer_fast([txt], max_length=1024, padding="max_length", truncation=True)

In [178]:
node_encodings = get_USKG_node_encodings(sample, model, tokenizer_fast, debug=True)

encoder_output_hidden_states: (1024, 1024)
node_pieces_ranges: [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (16, 19), (23, 27), (29, 30), (32, 36), (38, 39), (44, 49), (51, 53), (55, 57), (59, 60), (62, 66), (70, 71), (73, 76), (78, 81), (83, 86), (20, 21), (40, 42), (67, 68)]


In [179]:
node_encodings[0].shape

(1024,)

## Load data collectors from py scripts

### Data samples load

In [17]:
## load dataset

with open(dataset_path, 'r') as f:
    orig_dataset = json.load(f)
    
for d in orig_dataset:
    d['rat_sql_graph']['relations'] = json.loads(d['rat_sql_graph']['relations'])

len(orig_dataset), orig_dataset[0].keys()

(8421, dict_keys(['phase', 'table_id', 'question', 'sql', 'rat_sql_graph']))

In [18]:
orig_dataset[0]['question'], orig_dataset[0]['table_id']

('What position does the player who played for butler cc (ks) play?',
 '1-10015132-11')

## Probing: link prediction

### Sample test

In [10]:
## build collector

PROJ_DIR='/home/yshao/Projects'
DATASET='wikisql'

args = SimpleNamespace()

args.orig_dataset_dir = f'{PROJ_DIR}/language/language/xsp/data/{DATASET}'
args.graph_dataset_dir = f'{PROJ_DIR}/SDR-analysis/data/{DATASET}'
args.probing_data_in_dir = f'{PROJ_DIR}/SDR-analysis/data/probing/text2sql/link_prediction/{DATASET}/uskg'
args.probing_data_out_dir = f'{PROJ_DIR}/SDR-analysis/data/probing/text2sql/link_prediction/{DATASET}/uskg-tmp'
args.model = 'hkunlp/from_all_T5_large_prefix_spider_with_cell_value2'
args.cfg = 'Salesforce/T5_large_prefix_spider_with_cell_value.cfg'


probe_data_collector = LinkPredictionDataCollector_USKG_wikisql(
    orig_dataset_dir=args.orig_dataset_dir,
    graph_dataset_dir=args.graph_dataset_dir,
    probing_data_in_dir=args.probing_data_in_dir,
    probing_data_out_dir=args.probing_data_out_dir,
)

probe_data_collector.orig_ds_list = ['dev']
probe_data_collector.prob_ds_list = ['train']
# probe_data_collector._start_idx = 490

# probe_data_collector.load_model(args)
probe_data_collector.model = model
probe_data_collector.tokenizer_fast = tokenizer_fast

# probe_data_collector.collect_all_probing_datasets()


In [11]:
orig_ds = 'dev'
prob_ds = 'train'
# dataset_path = f"/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/spider/{orig_ds}+ratsql_graph.json"
dataset_path = f"/home/yshao/Projects/SDR-analysis/data/wikisql/{orig_ds}+ratsql_graph.json"


In [13]:
paths_dict = probe_data_collector.get_paths_dict()
paths_dict

{'dev': {'orig_dataset_path': '/home/yshao/Projects/language/language/xsp/data/wikisql/dev.jsonl',
  'orig_tables_path': '/home/yshao/Projects/language/language/xsp/data/wikisql/dev.tables.jsonl',
  'db_path': '/home/yshao/Projects/language/language/xsp/data/wikisql/dev.db'}}

In [16]:
orig_tables_path = paths_dict[orig_ds]['orig_tables_path']
db_path = paths_dict[orig_ds]['db_path']

db_schemas_dict = probe_data_collector.get_schemas_dict(orig_tables_path, db_path)
len(db_schemas_dict)

orig_tables_path=/home/yshao/Projects/language/language/xsp/data/wikisql/dev.tables.jsonl; db_path=/home/yshao/Projects/language/language/xsp/data/wikisql/dev.db: not match the cache, recompute!


2716

In [26]:
## start link prediction data collection 

enc_repr = probe_data_collector.get_node_encodings(orig_dataset[0])
type(enc_repr)

list

In [29]:
graph_dict = orig_dataset[0]['rat_sql_graph']

X, y, pos = collect_link_prediction_samples(graph_dict,
                                           enc_repr,
                                           pos=None,
                                           max_rel_occ=1,
                                           debug=True)

Sampled output idx: 7
i = 14, j = 8
Nodes: ?, butler
Relation: 0 (-)
Repr vectors:
[0.06752008 0.02653399 0.13214661] ... [-0.02297289  0.00501938  0.03224115]
[-0.14800712  0.01252076 -0.26400036] ... [-0.20672157  0.01301077  0.12664273]
Combined vector:
[0.06752008 0.02653399 0.13214661] ... [4.748992e-03 6.530606e-05 4.083108e-03]
Label:
0


In [51]:
y

[['question', 'butler'],
 ['question', 'cc'],
 ['question', 'what'],
 ['question', '?'],
 ['question', 'the'],
 ['column', 'year', 'in', 'toronto'],
 ['column', 'school', 'or', 'club', 'team'],
 ['column', 'position'],
 ['column', 'nationality'],
 ['column', '*'],
 ['table', 'l']]

### Data collection (moved to script)

In [149]:
def extract_probing_samples_link_prediction_uskg(dataset_sample,
                                                 db_schemas_dict,
                                                 model,
                                                 tokenizer,
                                                 pos=None,
                                                 max_rel_occ=None,
                                                 debug=False):
    """
    Args:
        dataset_sample (Dict): a sample dict from spider dataset
        db_schemas_dict (Dict): db_id => db_schema, precomputed for all DBs (not used here)
        model (EncDec): the rat-sql model
        pos (List[Tuple]): the position pairs to use. If none, will randomly generate
        max_rel_occ (int): each relation occur at most this many times in each (original) sample
    
    Return:
        X (List[np.array]): input features, "shape" = (n, dim)
        y (List): output labels, "shape" = (n,)
        pos (List[Tuple]): actual position (node-id) pairs for X and y
    """
    
    d = dataset_sample
    
    db_id = d['db_id']
    # db_schema = db_schemas_dict[db_id]
    question = d['question']

    # get relation matrix (relation_id2name not available as it needs rat-sql model)
    graph_dict = dataset_sample['rat_sql_graph']
    # graph_dict['relation_id2name'] = {v : k for k, v in model.encoder.encs_update.relation_ids.items()}
    
    # get encodings
    # rat_sql_encoder_state = get_rat_sql_encoder_state(question=question, db_schema=db_schema, model=model)
    # enc_repr = rat_sql_encoder_state.memory.squeeze(0).detach().cpu().numpy()
    enc_repr = get_USKG_node_encodings(sample=dataset_sample,
                                       model=model,
                                       tokenizer=tokenizer,
                                       debug=debug)
    
    X, y, pos = collect_link_prediction_samples(
        graph_dict,
        enc_repr,
        pos=pos,
        max_rel_occ=max_rel_occ,
        debug=debug)
    
    return X, y, pos



In [187]:
probing_data_dir = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/probing/text2sql/link_prediction/spider/ratsql"

orig_ds = 'dev'
prob_ds = 'test'
dataset_path = f"/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/spider/{orig_ds}+ratsql_graph.json"

pos_file_path = os.path.join(probing_data_dir, f'{orig_ds}.{prob_ds}.pos.txt')

In [188]:
with open(dataset_path, 'r') as f:
    orig_dataset = json.load(f)
    
for d in orig_dataset:
    d['rat_sql_graph']['relations'] = json.loads(d['rat_sql_graph']['relations'])

len(orig_dataset), orig_dataset[0].keys()

(1034,
 dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'rat_sql_graph']))

In [189]:
with open(pos_file_path, 'r') as f:
    lines = f.read().strip().split('\n')
    all_pos_triplets = [tuple([int(s) for s in l.split('\t')]) for l in lines]
len(all_pos_triplets), all_pos_triplets[0]

# Load pos file 
sample_ds_indices = []               # [ds_idx], based on occurring order 
pos_per_sample = defaultdict(list)   # key = ds_idx, value = pos_list: List[(i, j)]

for ds_idx, i, j in all_pos_triplets:
    if not sample_ds_indices or sample_ds_indices[-1] != ds_idx:
        sample_ds_indices.append(ds_idx)
    pos_per_sample[ds_idx].append((i, j))

len(sample_ds_indices), len(pos_per_sample)

(500, 500)

In [190]:
# test loading pos file 
set(sample_ds_indices) == set(pos_per_sample.keys())

True

In [180]:
# TODO: for each sample_ds_idx, get the sample & pos_list, get the X, y, pos; save. 

all_X = []
all_y = []
all_pos = []

for sample_ds_idx in tqdm(sample_ds_indices):
    dataset_sample = orig_dataset[sample_ds_idx]
    pos_list = pos_per_sample[sample_ds_idx]

    X, y, pos = extract_probing_samples_link_prediction_uskg(dataset_sample=dataset_sample,
                                                             db_schemas_dict=None,
                                                             model=model,
                                                             tokenizer=tokenizer_fast,
                                                             pos=pos_list,
                                                             max_rel_occ=None,  # when given pos, this is not needed 
                                                             debug=False)
    
    all_X.extend(X)
    all_y.extend(y)
    pos = [(sample_ds_idx, i, j) for i, j in pos]   # add sample idx 
    all_pos.extend(pos)
    
    time.sleep(0.5)

len(all_X), len(all_y), len(all_pos)

  0%|          | 0/500 [00:00<?, ?it/s]

(16059, 16059, 16059)

In [155]:
sample_ds_idx

209

In [183]:
probing_data_out_dir = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/probing/text2sql/link_prediction/spider/uskg"
os.makedirs(probing_data_out_dir, exist_ok=True)

output_path_test_X = os.path.join(probing_data_out_dir, f'{orig_ds}.{prob_ds}.X.pkl')
output_path_test_y = os.path.join(probing_data_out_dir, f'{orig_ds}.{prob_ds}.y.pkl')
output_path_test_pos = os.path.join(probing_data_out_dir, f'{orig_ds}.{prob_ds}.pos.txt')

with open(output_path_test_X, 'wb') as f:
    pickle.dump(all_X, f)
with open(output_path_test_y, 'wb') as f:
    pickle.dump(all_y, f)
with open(output_path_test_pos, 'w') as f:
    for idx, i, j in all_pos:
        f.write(f'{idx}\t{i}\t{j}\n')

## Probing: single node reconstruction

In [58]:
import sdr_analysis.helpers.single_node_reconstruction_collector
importlib.reload(sdr_analysis.helpers.single_node_reconstruction_collector)
from sdr_analysis.helpers.single_node_reconstruction_collector import collect_single_node_reconstruction_samples

### Sample test

In [77]:
## build collector 

PROJ_DIR='/home/yshao/Projects'
DATASET='wikisql'

args = SimpleNamespace()

args.orig_dataset_dir = f'{PROJ_DIR}/language/language/xsp/data/{DATASET}'
args.graph_dataset_dir = f'{PROJ_DIR}/SDR-analysis/data/{DATASET}'
args.probing_data_in_dir = f'{PROJ_DIR}/SDR-analysis/data/probing/text2sql/single_node_reconstruction/{DATASET}/uskg'
args.probing_data_out_dir = f'{PROJ_DIR}/SDR-analysis/data/probing/text2sql/single_node_reconstruction/{DATASET}/uskg-tmp'
args.model = 'hkunlp/from_all_T5_large_prefix_spider_with_cell_value2'
args.cfg = 'Salesforce/T5_large_prefix_spider_with_cell_value.cfg'


SNR_probe_data_collector = SingleNodeReconstructionDataCollector_USKG_wikisql(
    orig_dataset_dir=args.orig_dataset_dir,
    graph_dataset_dir=args.graph_dataset_dir,
    probing_data_in_dir=args.probing_data_in_dir,
    probing_data_out_dir=args.probing_data_out_dir,
)

SNR_probe_data_collector.orig_ds_list = ['dev']
SNR_probe_data_collector.prob_ds_list = ['train']
# SNR_probe_data_collector._start_idx = 490

# SNR_probe_data_collector.load_model(args)
SNR_probe_data_collector.model = model
SNR_probe_data_collector.tokenizer_fast = tokenizer_fast

# SNR_probe_data_collector.collect_all_probing_datasets()

In [78]:
orig_ds = 'dev'
prob_ds = 'train'
# dataset_path = f"/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/spider/{orig_ds}+ratsql_graph.json"
dataset_path = f"/home/yshao/Projects/SDR-analysis/data/wikisql/{orig_ds}+ratsql_graph.json"


In [79]:
paths_dict = SNR_probe_data_collector.get_paths_dict()
paths_dict

{'dev': {'orig_dataset_path': '/home/yshao/Projects/language/language/xsp/data/wikisql/dev.jsonl',
  'orig_tables_path': '/home/yshao/Projects/language/language/xsp/data/wikisql/dev.tables.jsonl',
  'db_path': '/home/yshao/Projects/language/language/xsp/data/wikisql/dev.db'}}

In [80]:
orig_tables_path = paths_dict[orig_ds]['orig_tables_path']
db_path = paths_dict[orig_ds]['db_path']

db_schemas_dict = SNR_probe_data_collector.get_schemas_dict(orig_tables_path, db_path)
len(db_schemas_dict)

orig_tables_path=/home/yshao/Projects/language/language/xsp/data/wikisql/dev.tables.jsonl; db_path=/home/yshao/Projects/language/language/xsp/data/wikisql/dev.db: not match the cache, recompute!


2716

In [82]:
_sample = orig_dataset[0]

enc_repr = SNR_probe_data_collector.get_node_encodings(_sample, pooling_func=lambda x: x)

type(enc_repr), type(enc_repr[0]), type(enc_repr[0][0])

(list, list, numpy.ndarray)

In [83]:
print([e.shape for e in enc_repr[0]])

[(1, 1024), (1, 1024), (1, 1024), (1, 1024), (1, 1024), (1, 1024), (1, 1024), (1, 1024), (2, 1024), (3, 1024), (1, 1024), (2, 1024), (1, 1024), (1, 1024), (1, 1024), (5, 1024), (1, 1024), (1, 1024), (2, 1024), (1, 1024), (7, 1024), (19, 1024), (2, 1024)]


In [34]:
graph_dict = _sample['rat_sql_graph']

In [128]:
_sample['txt_pieces']['txt']

'What position does the player who played for butler cc (ks) play?; structed knowledge:  | 1-10015132-11 | l : player , no , nationality , position , years_in_toronto , school_or_club_team ( butler cc (ks) )'

In [35]:
X, y, pos = collect_single_node_reconstruction_samples(graph_dict,
                                                       enc_repr,
                                                       pos=None,
                                                       max_node_type_occ=None,
                                                       debug=True)

Sample idx: 3
node_id = 3
Node: ['question', 'the'] (the)
Repr vectors:
[-0.10546452 -0.16901198 -0.06285214] ... [-0.17854233 -0.3109485   0.19226295]


In [36]:
list(zip(pos, y))

[(0, ['question', 'what']),
 (1, ['question', 'position']),
 (2, ['question', 'do']),
 (3, ['question', 'the']),
 (4, ['question', 'player']),
 (5, ['question', 'who']),
 (6, ['question', 'play']),
 (7, ['question', 'for']),
 (8, ['question', 'butler']),
 (9, ['question', 'cc']),
 (10, ['question', '-lrb-']),
 (11, ['question', 'k']),
 (12, ['question', '-rrb-']),
 (13, ['question', 'play']),
 (14, ['question', '?']),
 (16, ['column', 'player']),
 (17, ['column', 'no']),
 (18, ['column', 'nationality']),
 (19, ['column', 'position']),
 (20, ['column', 'year', 'in', 'toronto']),
 (21, ['column', 'school', 'or', 'club', 'team']),
 (22, ['table', 'l'])]

In [37]:
[x[:, :3] for x in X]

[array([[0.16765554, 0.19590843, 0.1182106 ]], dtype=float32),
 array([[0.18989907, 0.15153025, 0.3066842 ]], dtype=float32),
 array([[ 0.18852772, -0.14816955,  0.06566472]], dtype=float32),
 array([[-0.10546452, -0.16901198, -0.06285214]], dtype=float32),
 array([[ 0.01387155, -0.1469807 , -0.04858986]], dtype=float32),
 array([[-0.09723643, -0.19441439, -0.3072489 ]], dtype=float32),
 array([[-0.0749758 , -0.15097   , -0.29927436]], dtype=float32),
 array([[-0.02502756,  0.18236493, -0.02920943]], dtype=float32),
 array([[-0.15982388, -0.1592177 , -0.22788137],
        [-0.13619037,  0.18425922, -0.30011937]], dtype=float32),
 array([[ 0.06221096, -0.00930876, -0.18367842],
        [ 0.15632832,  0.10616314, -0.2654981 ],
        [-0.1114219 , -0.06422008, -0.43678653]], dtype=float32),
 array([[ 0.0325256 , -0.08600596, -0.11446463]], dtype=float32),
 array([[-0.09835997, -0.18946472,  0.01884983],
        [-0.02021622, -0.3451298 , -0.0703741 ]], dtype=float32),
 array([[ 0.114689

In [84]:
# test collector pipeline (calling extract_probing_samples_single_node_reconstruction() )

X, y, pos = SNR_probe_data_collector.extract_probing_samples_single_node_reconstruction([_sample], pos_list=[pos])

In [85]:
list(zip(pos, y)), [x[:, :3] for x in X]

([((0, 0), ['question', 'what']),
  ((0, 1), ['question', 'position']),
  ((0, 2), ['question', 'do']),
  ((0, 3), ['question', 'the']),
  ((0, 4), ['question', 'player']),
  ((0, 5), ['question', 'who']),
  ((0, 6), ['question', 'play']),
  ((0, 7), ['question', 'for']),
  ((0, 8), ['question', 'butler']),
  ((0, 9), ['question', 'cc']),
  ((0, 10), ['question', '-lrb-']),
  ((0, 11), ['question', 'k']),
  ((0, 12), ['question', '-rrb-']),
  ((0, 13), ['question', 'play']),
  ((0, 14), ['question', '?']),
  ((0, 16), ['column', 'player']),
  ((0, 17), ['column', 'no']),
  ((0, 18), ['column', 'nationality']),
  ((0, 19), ['column', 'position']),
  ((0, 20), ['column', 'year', 'in', 'toronto']),
  ((0, 21), ['column', 'school', 'or', 'club', 'team']),
  ((0, 22), ['table', 'l'])],
 [array([[0.16765554, 0.19590843, 0.1182106 ]], dtype=float32),
  array([[0.18989907, 0.15153025, 0.3066842 ]], dtype=float32),
  array([[ 0.18852772, -0.14816955,  0.06566472]], dtype=float32),
  array([[-0.

## KBQA - WebQSP

In [244]:
from seq2seq_construction import webqsp_sr

In [245]:
def webqsp_get_sample(ex):
    _ents = [[elm['text'], elm['kb_id']] for elm in ex["entities"]]
    _answers = [[elm['text'], elm['kb_id']] for elm in ex["answers"]]
    return {
        # "id": ex["ID"],
        "id": ex["id"],
        "question": ex["question"],
        # "answers": ex["answers"],
        "answers": _answers,
        # "s_expression": ex["s_expression"],
        # "kg_tuples": ex["kg_tuples"],
        "subgraph_tuples": ex["subgraph"]["tuples"],
        # "entities": ex["entities"],
        "entities": _ents,
        # "paths": ex["paths"],
    }

In [290]:
webqsp_path = '/home/yshao/Projects/SDR-analysis/data/webqsp/train_simple.jsonl'

In [297]:
webqsp_path = '/home/yshao/Projects/SDR-analysis/data/webqsp/train_simple_short.jsonl'

In [298]:
with open(webqsp_path, 'r') as f:
    webqsp_dataset_orig = [json.loads(l) for l in f]
    webqsp_dataset = [webqsp_get_sample(ex) for ex in webqsp_dataset_orig]
len(webqsp_dataset)

2848

In [299]:
d = webqsp_dataset[624]
d

{'id': 'WebQTrn-837',
 'question': 'where did jennifer hudson go to school',
 'answers': [['Langston University', 'j'],
  ['Dunbar Vocational High School', 'i']],
 'subgraph_tuples': [['a', 'music.artist.origin', 'd'],
  ['a', 'people.person.places_lived', 'g'],
  ['c', 'people.person.education', 'b'],
  ['a', 'people.person.education', 'e'],
  ['e', 'education.education.institution', 'j'],
  ['g', 'people.place_lived.location', 'h'],
  ['e', 'education.education.student', 'a'],
  ['d', 'authority.stanford.place', 'Chicago'],
  ['f', 'education.education.student', 'a'],
  ['d', 'location.hud_county_place.place', 'd'],
  ['a', 'people.person.education', 'f'],
  ['f', 'type.object.type', 'education.education'],
  ['e', 'type.object.type', 'education.education'],
  ['d', 'authority.hud.countyplace', '171296'],
  ['f', 'education.education.institution', 'i'],
  ['a', 'influence.influence_node.influenced_by', 'c']],
 'entities': [['Jennifer Hudson', 'a']]}

### entities in subgraph

In [277]:
subg_entities_counter = Counter()

for i, d in enumerate(webqsp_dataset_orig):
    ent_ids = [e for e in d['subgraph']['entities'] if e.startswith('m.')]
    n_ents = len(ent_ids)
    if n_ents > 1000:
        print(i, n_ents)
    subg_entities_counter[n_ents] += 1

158 1238
228 1012
504 1002
755 1004
821 1059
879 1034
913 1012
2044 1012
2205 1005
2222 1058


In [280]:
cnt_26 = 0
cnt_676 = 0

for l, cnt in subg_entities_counter.items():
    if l < 26:
        cnt_26 += cnt
        cnt_676 += cnt
    elif l < 676:
        cnt_676 += cnt

cnt_26, cnt_676, len(webqsp_dataset_orig)

(1614, 2817, 2848)

### linearized input length

In [285]:
tokenizer_fast = AutoTokenizer.from_pretrained('t5-large', use_fast=True)

In [300]:
q, struct_in = webqsp_sr.kgqa_get_input(d['question'], d['subgraph_tuples'], d['entities'])
q, struct_in

('where did jennifer hudson go to school',
 'Jennifer Hudson: a | a music.artist.origin d | a people.person.places_lived g | c people.person.education b | a people.person.education e | e education.education.institution j | g people.place_lived.location h | e education.education.student a | d authority.stanford.place Chicago | f education.education.student a | d location.hud_county_place.place d | a people.person.education f | f type.object.type education.education | e type.object.type education.education | d authority.hud.countyplace 171296 | f education.education.institution i | a influence.influence_node.influenced_by c')

In [301]:
lin_input = "{}; structed knowledge: {}".format(q, struct_in)
lin_input

'where did jennifer hudson go to school; structed knowledge: Jennifer Hudson: a | a music.artist.origin d | a people.person.places_lived g | c people.person.education b | a people.person.education e | e education.education.institution j | g people.place_lived.location h | e education.education.student a | d authority.stanford.place Chicago | f education.education.student a | d location.hud_county_place.place d | a people.person.education f | f type.object.type education.education | e type.object.type education.education | d authority.hud.countyplace 171296 | f education.education.institution i | a influence.influence_node.influenced_by c'

In [302]:
len(tokenizer_fast.tokenize(lin_input))

221

In [303]:
tokenizer_fast.tokenize(lin_input)

['▁where',
 '▁did',
 '▁je',
 'n',
 'n',
 'if',
 'er',
 '▁',
 'h',
 'u',
 'd',
 'son',
 '▁go',
 '▁to',
 '▁school',
 ';',
 '▁',
 'struct',
 'e',
 'd',
 '▁knowledge',
 ':',
 '▁Jennifer',
 '▁Hudson',
 ':',
 '▁',
 'a',
 '▁|',
 '▁',
 'a',
 '▁music',
 '.',
 'art',
 'ist',
 '.',
 'o',
 'rig',
 'in',
 '▁',
 'd',
 '▁|',
 '▁',
 'a',
 '▁people',
 '.',
 'person',
 '.',
 'place',
 's',
 '_',
 'live',
 'd',
 '▁',
 'g',
 '▁|',
 '▁',
 'c',
 '▁people',
 '.',
 'person',
 '.',
 'education',
 '▁',
 'b',
 '▁|',
 '▁',
 'a',
 '▁people',
 '.',
 'person',
 '.',
 'education',
 '▁',
 'e',
 '▁|',
 '▁',
 'e',
 '▁education',
 '.',
 'education',
 '.',
 'in',
 'stitution',
 '▁',
 'j',
 '▁|',
 '▁',
 'g',
 '▁people',
 '.',
 'place',
 '_',
 'live',
 'd',
 '.',
 'location',
 '▁',
 'h',
 '▁|',
 '▁',
 'e',
 '▁education',
 '.',
 'education',
 '.',
 'stud',
 'ent',
 '▁',
 'a',
 '▁|',
 '▁',
 'd',
 '▁authority',
 '.',
 'stan',
 'ford',
 '.',
 'place',
 '▁Chicago',
 '▁|',
 '▁',
 'f',
 '▁education',
 '.',
 'education',
 '.',
 'st

In [ ]:
tokenized_txt = tokenizer_fast(lin_input, truncation=True, padding="max_length", max_length=1024)

In [ ]:
len(tokenized_txt.data['input_ids'])

In [293]:
t5_len_counter = Counter()

for d in webqsp_dataset:
    q, struct_in = webqsp_sr.kgqa_get_input(d['question'], d['subgraph_tuples'], d['entities'])
    lin_input = "{}; structed knowledge: {}".format(q, struct_in)
    t5_len = len(tokenizer_fast.tokenize(lin_input))
    t5_len_counter[t5_len] += 1

In [294]:
# original SR
cnt_512 = 0
cnt_1024 = 0

for l, cnt in t5_len_counter.items():
    if l < 512:
        cnt_512 += cnt
        cnt_1024 += cnt
    elif l < 1024:
        cnt_1024 += cnt

cnt_512, cnt_1024, len(webqsp_dataset)

(766, 1555, 2848)

In [289]:
# shorter
cnt_512 = 0
cnt_1024 = 0

for l, cnt in t5_len_counter.items():
    if l < 512:
        cnt_512 += cnt
        cnt_1024 += cnt
    elif l < 1024:
        cnt_1024 += cnt

cnt_512, cnt_1024, len(webqsp_dataset)

(1346, 1985, 2848)

## Temp

### Test tokenizer

In [108]:
tokenizer_fast = AutoTokenizer.from_pretrained('t5-base', use_fast=True)

In [ ]:
test_txt = "This is t5's tokenization.; structed knowledge: | model | plm(t5), rnn"

In [128]:
tok_fast = tokenizer_fast.tokenize(test_txt)
tok_fast

['▁This',
 '▁is',
 '▁',
 't',
 '5',
 "'",
 's',
 '▁token',
 'ization',
 '.',
 ';',
 '▁',
 'struct',
 'e',
 'd',
 '▁knowledge',
 ':',
 '▁|',
 '▁model',
 '▁|',
 '▁pl',
 'm',
 '(',
 't',
 '5)',
 ',',
 '▁',
 'r',
 'n',
 'n']

In [113]:
encoded = tokenizer.encode(test_txt)
encoded_fast = tokenizer_fast.encode(test_txt)
type(encoded), type(encoded_fast)

(list, list)

In [115]:
list(zip(encoded, encoded_fast))

[(100, 100),
 (19, 19),
 (3, 3),
 (17, 17),
 (755, 755),
 (31, 31),
 (7, 7),
 (14145, 14145),
 (1707, 1707),
 (5, 5),
 (117, 117),
 (3, 3),
 (7593, 7593),
 (15, 15),
 (26, 26),
 (1103, 1103),
 (10, 10),
 (1820, 1820),
 (825, 825),
 (1820, 1820),
 (4752, 4752),
 (51, 51),
 (599, 599),
 (17, 17),
 (9120, 9120),
 (6, 6),
 (3, 3),
 (52, 52),
 (29, 29),
 (29, 29),
 (1, 1)]

In [118]:
ed_fast = tokenizer_fast(test_txt)
type(ed_fast), ed_fast

(transformers.tokenization_utils_base.BatchEncoding,
 {'input_ids': [100, 19, 3, 17, 755, 31, 7, 14145, 1707, 5, 117, 3, 7593, 15, 26, 1103, 10, 1820, 825, 1820, 4752, 51, 599, 17, 9120, 6, 3, 52, 29, 29, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [133]:
len(ed_fast.data['input_ids'])

31

In [124]:
ed_fast.word_ids()

[0,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 5,
 6,
 7,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 None]

In [131]:
len(ed_fast.word_ids())

31

In [132]:
len(tok_fast)

30

In [135]:
for sw_id, w_id in enumerate(ed_fast.word_ids()[:-1]):
    # the last is eos
    sw = tok_fast[sw_id]
    w = test_txt.split(' ')[w_id]
    print(f'{sw:<15s}{w}')

▁This          This
▁is            is
▁              t5's
t              t5's
5              t5's
'              t5's
s              t5's
▁token         tokenization.;
ization        tokenization.;
.              tokenization.;
;              tokenization.;
▁              structed
struct         structed
e              structed
d              structed
▁knowledge     knowledge:
:              knowledge:
▁|             |
▁model         model
▁|             |
▁pl            plm(t5),
m              plm(t5),
(              plm(t5),
t              plm(t5),
5)             plm(t5),
,              plm(t5),
▁              rnn
r              rnn
n              rnn
n              rnn


In [143]:
ed_fast.word_to_tokens(3)

TokenSpan(start=7, end=11)

In [144]:
ed = tokenizer(test_txt)
type(ed), ed

(transformers.tokenization_utils_base.BatchEncoding,
 {'input_ids': [100, 19, 3, 17, 755, 31, 7, 14145, 1707, 5, 117, 3, 7593, 15, 26, 1103, 10, 1820, 825, 1820, 4752, 51, 599, 17, 9120, 6, 3, 52, 29, 29, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [ ]:
ed.word_ids()

#### Different tokenizers consistency

In [301]:
ds_path = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/spider/dev+ratsql_graph.json"

with open(ds_path, 'r') as f:
    dataset2 = json.load(f)

for d in dataset2:
    d['rat_sql_graph']['relations'] = json.loads(d['rat_sql_graph']['relations'])

len(dataset2)

1034

In [302]:
for i, d in enumerate(tqdm(dataset2)):
    text_in = d['question']
    struct_in = uskg_sample_to_struct_input(d)
    txt = "{}; structed knowledge: {}".format(text_in, struct_in)
    
    if "<" not in txt:
        continue
    
    uskg_tokenized_txt = tokenizer(txt, max_length=1024, padding="max_length", truncation=True)
    base_tokenized_txt = tokenizer_fast(txt, max_length=1024, padding="max_length", truncation=True)
    large_tokenized_txt = tokenizer_large_fast(txt, max_length=1024, padding="max_length", truncation=True)
    
    assert uskg_tokenized_txt.data == base_tokenized_txt.data == large_tokenized_txt.data, (i, txt)
    

  0%|          | 0/1034 [00:00<?, ?it/s]

In [288]:
vocab_uskg = tokenizer.get_vocab()
vocab_base = tokenizer_fast.get_vocab()
vocab_large = tokenizer_large_fast.get_vocab()

In [298]:
for k in set(vocab_uskg.keys()) | set(vocab_base.keys()) | set(vocab_large.keys()):
    v = vocab_uskg.get(k, None)
    v1 = vocab_base.get(k, None)
    v2 = vocab_large.get(k, None)
    if not v == v1 == v2:
        print(k, v, v1, v2)
#     assert v == v2, (k, v, v2)

 <= 32101 None None
 < 32100 None None


In [295]:
vocab_uskg[' <']

32100

In [303]:
for i, d in enumerate(tqdm(dataset2)):
    text_in = d['question']
    struct_in = uskg_sample_to_struct_input(d)
    txt = "{}; structed knowledge: {}".format(text_in, struct_in)
    
    if "<" in txt:
        print(i, txt)

  0%|          | 0/1034 [00:00<?, ?it/s]

#### max length

In [305]:
idx = 777
text_in = dataset2[idx]['question']
struct_in = uskg_sample_to_struct_input(d)
txt = "{}; structed knowledge: {}".format(text_in, struct_in)
txt

'What are the Asian countries which have a population larger than that of any country in Africa?; structed knowledge:  | real_estate_properties | ref_feature_types : feature_type_code , feature_type_name | ref_property_types : property_type_code ( Apartment , House ) , property_type_description | other_available_features : feature_id , feature_type_code , feature_name , feature_description | properties : property_id , property_type_code ( Apartment , House ) , date_on_market , date_sold , property_name , property_address , room_count , vendor_requested_price , buyer_offered_price , agreed_selling_price , apt_feature_1 , apt_feature_2 , apt_feature_3 , fld_feature_1 , fld_feature_2 , fld_feature_3 , hse_feature_1 , hse_feature_2 , hse_feature_3 , oth_feature_1 , oth_feature_2 , oth_feature_3 , shp_feature_1 , shp_feature_2 , shp_feature_3 , other_property_details | other_property_features : property_id , feature_id , property_feature_description'

In [308]:
_tkn_txt = tokenizer_fast(txt, max_length=32, truncation=True)

In [316]:
for i in range(31):
    st, ed = _tkn_txt.token_to_chars(i)
    print(txt[st:ed])

What
are
the
Asian
countries
which
have
a
a
population
larger
than
that
of
any
country
in
Africa
?
;
s
struct
e
d
knowledge
:
|
real
_
e
state


In [318]:
list(enumerate(txt.split()))

[(0, 'What'),
 (1, 'are'),
 (2, 'the'),
 (3, 'Asian'),
 (4, 'countries'),
 (5, 'which'),
 (6, 'have'),
 (7, 'a'),
 (8, 'population'),
 (9, 'larger'),
 (10, 'than'),
 (11, 'that'),
 (12, 'of'),
 (13, 'any'),
 (14, 'country'),
 (15, 'in'),
 (16, 'Africa?;'),
 (17, 'structed'),
 (18, 'knowledge:'),
 (19, '|'),
 (20, 'real_estate_properties'),
 (21, '|'),
 (22, 'ref_feature_types'),
 (23, ':'),
 (24, 'feature_type_code'),
 (25, ','),
 (26, 'feature_type_name'),
 (27, '|'),
 (28, 'ref_property_types'),
 (29, ':'),
 (30, 'property_type_code'),
 (31, '('),
 (32, 'Apartment'),
 (33, ','),
 (34, 'House'),
 (35, ')'),
 (36, ','),
 (37, 'property_type_description'),
 (38, '|'),
 (39, 'other_available_features'),
 (40, ':'),
 (41, 'feature_id'),
 (42, ','),
 (43, 'feature_type_code'),
 (44, ','),
 (45, 'feature_name'),
 (46, ','),
 (47, 'feature_description'),
 (48, '|'),
 (49, 'properties'),
 (50, ':'),
 (51, 'property_id'),
 (52, ','),
 (53, 'property_type_code'),
 (54, '('),
 (55, 'Apartment'),

In [320]:
st, ed = _tkn_txt.word_to_chars(20)
st, ed, txt[st:ed]

(120, 131, 'real_estate')

In [322]:
_tkn_txt.word_to_chars(21)

TypeError: type object argument after * must be an iterable, not NoneType

#### Unk

In [75]:
_txt = "question \u516b\u5c3e"
_txt

'question 八尾'

In [78]:
_tkn_txt = tokenizer_fast.tokenize(_txt)
_tkn_txt

['▁question', '▁', '八尾']

In [79]:
_tkn_txt_ids = tokenizer_fast.convert_tokens_to_ids(_tkn_txt)
_tkn_txt_recon = tokenizer_fast.convert_ids_to_tokens(_tkn_txt_ids)
_tkn_txt_ids, _tkn_txt_recon

([822, 3, 2], ['▁question', '▁', '<unk>'])

In [81]:
t5_vocab = tokenizer_fast.get_vocab()
len(t5_vocab)

32100

In [82]:
tokenizer_fast.unk_token_id

2

### Test get encoding

In [60]:
type(model.pretrain_model.encoder)

models.prompt.modeling_t5.T5Stack

In [61]:
# pred = tokenizer.batch_decode(
#   model.generate(
#     torch.LongTensor(tokenized_txt.data['input_ids']),
#     torch.LongTensor(tokenized_txt.data['attention_mask']),
#     num_beams=1, 
#     max_length=256
#     ), 
#   skip_special_tokens=True 
# )

In [62]:
# generated_ids = self.pretrain_model.generate(
#     input_ids=input_ids,
#     attention_mask=attention_mask,
#     past_prompt=past_prompt,
#     use_cache=True,
#     **kwargs,
# )

# model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)

# outputs = self(
#     **model_inputs,
#     return_dict=True,
#     output_attentions=output_attentions,
#     output_hidden_states=output_hidden_states,
# )

In [64]:
past_prompt = model.get_prompt(
    bsz=1,              # bsz = input_ids.shape[0]
    sample_size=1,      # sample_size=kwargs['num_beams']
    description=None,   
    knowledge=None,     
)

In [65]:
encoder_outputs = model.pretrain_model.encoder(
    input_ids=torch.LongTensor(tokenized_txt.data['input_ids']),
    attention_mask=torch.LongTensor(tokenized_txt.data['attention_mask']),
    past_prompt=past_prompt,
)

In [66]:
past_prompt[0].keys()

dict_keys(['decoder_prompt', 'cross_attention_prompt', 'encoder_prompt'])

In [67]:
encoder_outputs.__dict__.keys()

dict_keys(['last_hidden_state', 'past_key_values', 'hidden_states', 'attentions', 'cross_attentions'])

In [68]:
encoder_outputs.last_hidden_state.size()

torch.Size([1, 1024, 1024])

In [69]:
sum(tokenized_txt.data['attention_mask'][0])

158

In [70]:
# has zero-padding and EOS, no BOS  
len(tokenized_txt.data['input_ids'][0]), np.greater(tokenized_txt.data['input_ids'][0], 0).sum()

(1024, 158)

### Tokenized pieces-nodes mapping (old)

In [ ]:
tokenized_txt.data['input_ids'][0][:150]

In [213]:
len(tokenizer.tokenize(txt))

157

In [214]:
tokenizer.tokenize(txt)

['▁What',
 '▁are',
 '▁the',
 '▁Asian',
 '▁countries',
 '▁which',
 '▁have',
 '▁',
 'a',
 '▁population',
 '▁larger',
 '▁than',
 '▁that',
 '▁of',
 '▁any',
 '▁country',
 '▁in',
 '▁Africa',
 '?',
 ';',
 '▁',
 'struct',
 'e',
 'd',
 '▁knowledge',
 ':',
 '▁|',
 '▁world',
 '_',
 '1',
 '▁|',
 '▁city',
 '▁',
 ':',
 '▁',
 'i',
 'd',
 '▁',
 ',',
 '▁name',
 '▁',
 ',',
 '▁country',
 'code',
 '▁',
 ',',
 '▁district',
 '▁',
 ',',
 '▁population',
 '▁|',
 '▁sq',
 'lite',
 '_',
 's',
 'e',
 'que',
 'nce',
 '▁',
 ':',
 '▁name',
 '▁',
 ',',
 '▁se',
 'q',
 '▁|',
 '▁country',
 '▁',
 ':',
 '▁code',
 '▁',
 ',',
 '▁name',
 '▁',
 ',',
 '▁continent',
 '▁(',
 '▁Africa',
 '▁',
 ',',
 '▁Asia',
 '▁',
 ')',
 '▁',
 ',',
 '▁region',
 '▁',
 ',',
 '▁surface',
 'area',
 '▁',
 ',',
 '▁in',
 'de',
 'p',
 'year',
 '▁',
 ',',
 '▁population',
 '▁',
 ',',
 '▁life',
 'ex',
 'pe',
 'c',
 't',
 'ancy',
 '▁',
 ',',
 '▁',
 'g',
 'n',
 'p',
 '▁',
 ',',
 '▁',
 'g',
 'n',
 'pol',
 'd',
 '▁',
 ',',
 '▁local',
 'name',
 '▁',
 ',',
 '▁gove

In [215]:
' '.join(sample['rat_sql_graph']['nodes']), txt

('what be the asian country which have a population larger than that of any country in africa ? <C>NONE::* <C>city::id <C>city::name <C>city::countrycode <C>city::district <C>city::population <C>sqlite_sequence::name <C>sqlite_sequence::seq <C>country::code <C>country::name <C>country::continent <C>country::region <C>country::surfacearea <C>country::indepyear <C>country::population <C>country::lifeexpectancy <C>country::gnp <C>country::gnpold <C>country::localname <C>country::governmentform <C>country::headofstate <C>country::capital <C>country::code2 <C>countrylanguage::countrycode <C>countrylanguage::language <C>countrylanguage::isofficial <C>countrylanguage::percentage <T>city <T>sqlite_sequence <T>country <T>countrylanguage',
 'What are the Asian countries which have a population larger than that of any country in Africa?; structed knowledge:  | world_1 | city : id , name , countrycode , district , population | sqlite_sequence : name , seq | country : code , name , continent ( Afri

In [216]:
' '.join(sample['question_toks'])

'What are the Asian countries which have a population larger than that of any country in Africa ?'

In [218]:
type(tokenized_txt)

transformers.tokenization_utils_base.BatchEncoding

#### Nodes to char ranges

In [219]:
# ratsql nodes to sentence char ids 

ratsql_graph_nodes = sample['rat_sql_graph']['nodes']
question_toks = sample['question_toks']

_q_nodes = []  # [stem token (node name)]
q_nodes = []  # [(stem token (node name), orig question token)]
c_nodes = []  # [(orig table name, orig column name)]
t_nodes = []  # [orig table name]

for n in ratsql_graph_nodes:
    if n.startswith('<C>'):
        _n = n[3:]
        _t, _c = _n.split('::')
        c_nodes.append((_t, _c))
    elif n.startswith('<T>'):
        _n = n[3:]
        t_nodes.append(_n)
    else:
        _q_nodes.append(n)

assert len(_q_nodes) == len(question_toks), (_q_nodes, question_toks)
q_nodes = list(zip(_q_nodes, question_toks))

q_nodes, c_nodes, t_nodes

([('what', 'What'),
  ('be', 'are'),
  ('the', 'the'),
  ('asian', 'Asian'),
  ('country', 'countries'),
  ('which', 'which'),
  ('have', 'have'),
  ('a', 'a'),
  ('population', 'population'),
  ('larger', 'larger'),
  ('than', 'than'),
  ('that', 'that'),
  ('of', 'of'),
  ('any', 'any'),
  ('country', 'country'),
  ('in', 'in'),
  ('africa', 'Africa'),
  ('?', '?')],
 [('NONE', '*'),
  ('city', 'id'),
  ('city', 'name'),
  ('city', 'countrycode'),
  ('city', 'district'),
  ('city', 'population'),
  ('sqlite_sequence', 'name'),
  ('sqlite_sequence', 'seq'),
  ('country', 'code'),
  ('country', 'name'),
  ('country', 'continent'),
  ('country', 'region'),
  ('country', 'surfacearea'),
  ('country', 'indepyear'),
  ('country', 'population'),
  ('country', 'lifeexpectancy'),
  ('country', 'gnp'),
  ('country', 'gnpold'),
  ('country', 'localname'),
  ('country', 'governmentform'),
  ('country', 'headofstate'),
  ('country', 'capital'),
  ('country', 'code2'),
  ('countrylanguage', 'count

In [220]:
txt

'What are the Asian countries which have a population larger than that of any country in Africa?; structed knowledge:  | world_1 | city : id , name , countrycode , district , population | sqlite_sequence : name , seq | country : code , name , continent ( Africa , Asia ) , region , surfacearea , indepyear , population , lifeexpectancy , gnp , gnpold , localname , governmentform , headofstate , capital , code2 | countrylanguage : countrycode , language , isofficial , percentage'

In [222]:
# tokenized_txt = tokenizer_fast([txt], max_length=1024, padding="max_length", truncation=True)

In [223]:
# txt = "This is t5's tokenization.; structed knowledge: | model | plm(t5), rnn"

# tokenized_txt = tokenizer_fast([test_txt], max_length=1024, padding="max_length", truncation=True)

In [224]:
# number of whitespace words 
max([w_id for w_id in tokenized_txt.word_ids() if w_id is not None]) + 1, len(txt.split())

(86, 86)

In [230]:
_splitter = "; structed knowledge:  "  # struct_in has a preceding white space 
text_in, struct_in = txt.split(_splitter)

q_node_chars = []   # [(st, ed)]; same below
c_node_chars = []
t_node_chars = []

In [231]:
# Text part
# Assumption: the mismatch between whitespace words (text_words) and question words only come from trailing puncts

text_words = text_in.strip().split(' ') + ['<SENTINAL>']
text_word_char_ranges = [tokenized_txt.word_to_chars(i) for i in range(len(text_words) - 1)] + [(None, None)]  # -1 to remove the sentinal 

len(text_words), len(text_word_char_ranges), text_word_char_ranges[0]

(18, 18, CharSpan(start=0, end=4))

In [232]:
curr_tw_idx = 0
curr_tw = text_words[0]
curr_tw_char_range = text_word_char_ranges[0]
curr_char_ptr = 0

for stem_tok, orig_tok in q_nodes:
    if curr_tw == orig_tok:
        # finishing current word 
        q_node_chars.append((curr_char_ptr, curr_char_ptr + len(orig_tok)))   # curr pos to curr pos + len 
        curr_tw_idx += 1
        curr_tw = text_words[curr_tw_idx]
        curr_tw_char_range = text_word_char_ranges[curr_tw_idx]
        curr_char_ptr = curr_tw_char_range[0]
    else:
        # not finishing current word 
        assert curr_tw.startswith(orig_tok), (curr_tw, orig_tok)
        q_node_chars.append((curr_char_ptr, curr_char_ptr + len(orig_tok)))   # curr pos to curr pos + len 
        curr_char_ptr += len(orig_tok)     # move ptr forward by len 
        curr_tw = curr_tw[len(orig_tok):]  # get the remaining chars in the word 


In [233]:
[txt[st:ed] for st, ed in q_node_chars] == sample['question_toks']

True

In [235]:
# Struct part 
assert not struct_in.startswith(' ')
_str_before_struct = text_in + _splitter
_n_words_before_struct = len(_str_before_struct.strip().split(' '))

struct_words_st_idx = _n_words_before_struct
assert len(_str_before_struct) == tokenized_txt.word_to_chars(struct_words_st_idx)[0], \
    (len(_str_before_struct), tokenized_txt.word_to_chars(struct_words_st_idx)[0])   # len(before) == starting char idx 

In [236]:
struct_in

'| world_1 | city : id , name , countrycode , district , population | sqlite_sequence : name , seq | country : code , name , continent ( Africa , Asia ) , region , surfacearea , indepyear , population , lifeexpectancy , gnp , gnpold , localname , governmentform , headofstate , capital , code2 | countrylanguage : countrycode , language , isofficial , percentage'

In [287]:
struct_ranges_collector = StructCharRangesCollector()
struct_ranges_collector.collect(struct_in, tokenized_txt, _n_words_before_struct)

In [288]:
for db_id_name, (st, ed) in struct_ranges_collector.db_id2char_ranges.items():
    assert txt[st:ed] == db_id_name, (st, ed, txt[st:ed], db_id_name) 

for table_name, (st, ed) in struct_ranges_collector.table2char_ranges.items():
    assert txt[st:ed] == table_name, (st, ed, txt[st:ed], table_name) 

for (_, col_name), (st, ed) in struct_ranges_collector.column2char_ranges.items():
    txt_piece = txt[st:ed].split(' ( ')[0]
    assert txt_piece == col_name, (st, ed, txt[st:ed], col_name) 

In [289]:
# for db_id_name, (st, ed) in struct_ranges_collector.db_id2char_ranges.items():
#     print(st, ed, txt[st:ed], db_id_name) 
# print()
# for table_name, (st, ed) in struct_ranges_collector.table2char_ranges.items():
#     print(st, ed, txt[st:ed], table_name) 
# print()
# for (_, col_name), (st, ed) in struct_ranges_collector.column2char_ranges.items():
#     txt_piece = txt[st:ed].split(' ( ')[0]
#     print(st, ed, txt[st:ed], col_name) 

In [290]:
for c_node in c_nodes:
    if c_node == ('NONE', '*'):
        # the special column in spider, using db_id 
        c_node_chars.append(list(struct_ranges_collector.db_id2char_ranges.values())[0])   # assuming only 1 db_id, which should be true...
    else:
        c_node_chars.append(struct_ranges_collector.column2char_ranges[c_node])

for t_node in t_nodes:
    t_node_chars.append(struct_ranges_collector.table2char_ranges[t_node])

#### Nodes to tokenized pieces

In [78]:
node_char_ranges = char_ranges_dict['q_node_chars'] + char_ranges_dict['c_node_chars'] + char_ranges_dict['t_node_chars']
len(node_char_ranges), len(sample['rat_sql_graph']['nodes'])

(49, 49)

In [79]:
# some chars can be mapped to multiple tokens (e.g. 'i' => '▁', 'i' )
char_to_tokens_dict = defaultdict(list)

for token_idx, tok in enumerate(tokenized_txt.tokens()):
    if tok == '</s>':
        break
    char_span = tokenized_txt.token_to_chars(token_idx)
    for char_idx in range(char_span[0], char_span[1]):
        char_to_tokens_dict[char_idx].append(token_idx)

len(char_to_tokens_dict)

393

In [ ]:
node_pieces_ranges = []

for st, ed in node_ranges:
    piece_ids = []
    for char_idx in range(st, ed):
        _piece_ids = char_to_tokens_dict[char_idx]
        piece_ids.extend(_piece_ids)
    
    piece_st = piece_ids[0]
    piece_ed = piece_ids[-1] + 1
    # the collected piece_ids should be continuous 
    # ^ not true... some chars can be mapped to multiple tokens (started by ▁ )
    # re-collect a char-to-token
    assert set(range(piece_st, piece_ed)) == set(piece_ids), piece_ids
    
    node_pieces_ranges.append((piece_st, piece_ed))

In [ ]:
for n, (p_st, p_ed) in zip(sample['rat_sql_graph']['nodes'], node_pieces_ranges):
    print(n, '\t', tokenized_txt.tokens()[p_st:p_ed])

### data: server vs. local

In [197]:
local_data_path = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/probing/text2sql/link_prediction/spider/uskg/local/dev.train.X.pkl"
server_data_path = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/probing/text2sql/link_prediction/spider/uskg/dev.train.X.pkl"

with open(local_data_path, 'rb') as f:
    local_data_X = pickle.load(f)
with open(server_data_path, 'rb') as f:
    server_data_X = pickle.load(f)

type(local_data_X), type(server_data_X)

(list, list)

In [201]:
local_data_X = np.array(local_data_X)
server_data_X = np.array(server_data_X)
local_data_X.shape, server_data_X.shape

((16059, 3072), (16059, 3072))

In [211]:
large_diff_ids = np.argsort(np.max(np.abs(local_data_X - server_data_X), axis=1))[::-1]
large_diff_ids[:10]

array([ 5264,  5255,  5247,   288,   292,   294, 11989, 11979,  8755,
        8756])

In [216]:
ds_path = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/spider/dev+ratsql_graph.json"
pos_path = "/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SDR-analysis/data/probing/text2sql/link_prediction/spider/uskg/dev.train.pos.txt"

with open(ds_path, 'r') as f:
    ds_samples = json.load(f)
    for d in ds_samples:
        d['rat_sql_graph']['relations'] = json.loads(d['rat_sql_graph']['relations'])
with open(pos_path, 'r') as f:
    pos_lines = f.read().strip().split('\n')

len(ds_samples), len(pos_lines)

(1034, 16059)

In [258]:
_idx = large_diff_ids[51]
_idx

5245

In [259]:
# _idx = 8970
local_data_X[_idx][::300], server_data_X[_idx][::300], max(local_data_X[_idx] - server_data_X[_idx])

(array([-0.16274624, -0.06520639,  0.11448385,  0.02375873,  0.00876573,
        -0.10075585,  0.11073106,  0.00379219,  0.07125453,  0.01576799,
         0.01186033], dtype=float32),
 array([-0.17650707, -0.07676765,  0.13225155,  0.01257438,  0.00267248,
        -0.1266068 ,  0.12796973,  0.00241799,  0.05103843,  0.03357478,
         0.01657346], dtype=float32),
 0.07821107)

In [260]:
sid, i, j = [int(s) for s in pos_lines[_idx].split('\t')]
sid, i, j

(696, 6, 6)

In [261]:
_nodes = ds_samples[sid]['rat_sql_graph']['nodes']
_nodes

['what',
 'be',
 'the',
 'number',
 'of',
 'vote',
 'from',
 'state',
 '`',
 'ny',
 "'",
 'or',
 '`',
 'ca',
 "'",
 '?',
 '<C>NONE::*',
 '<C>area_code_state::area_code',
 '<C>area_code_state::state',
 '<C>contestant::contestant_number',
 '<C>contestant::contestant_name',
 '<C>vote::vote_id',
 '<C>vote::phone_number',
 '<C>vote::state',
 '<C>vote::contestant_number',
 '<C>vote::create',
 '<T>area_code_state',
 '<T>contestant',
 '<T>vote']

In [262]:
_nodes[i], _nodes[j]

('from', 'from')

### Model params

In [323]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take spider for example
            '--cfg', 'Salesforce/T5_base_finetune_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
tmp_args = Configure.Get(training_args.cfg)

In [330]:
tmp_args.bert.location

't5-base'

In [340]:
# model_path = 't5-base'
# model_path = 'hkunlp/from_all_T5_large_prefix_spider_with_cell_value2'
# model_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-asr_mixed/checkpoint-79500/'
# model_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-rewritten_mixed/checkpoint-56500/'

# tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# for reconstruction
# tokenizer_fast = AutoTokenizer.from_pretrained('t5-base', use_fast=True)

tmp_model = finetune.Model(tmp_args)

In [341]:
tmp_model.pretrain_model.encoder.block[0].layer[0].SelfAttention.q.weight[:3,:3]

tensor([[ 0.0762, -0.0471,  0.0309],
        [ 0.0381, -0.0075,  0.0003],
        [-0.0047, -0.0262, -0.0298]], grad_fn=<SliceBackward>)

In [342]:
# model_path = 't5-base'
# tmp_model.load(model_path)
# # need tmp_model.pretrained_model.load(...)?

In [343]:
tmp_model.pretrain_model.init_weights()

In [344]:
tmp_model.pretrain_model.encoder.block[0].layer[0].SelfAttention.q.weight[:3,:3]

tensor([[ 0.0013,  0.0083,  0.0015],
        [ 0.0008, -0.0017,  0.0039],
        [-0.0088, -0.0019, -0.0036]], grad_fn=<SliceBackward>)

In [88]:
from transformers import T5ForConditionalGeneration

t5_model_1 = T5ForConditionalGeneration.from_pretrained('t5-large')

In [91]:
t5_model_1.model_dim

1024

### Data inspection

#### Original dataset samples

In [250]:
orig_path = f"/home/yshao/Projects/SDR-analysis/data/wikisql/test+ratsql_graph.json"

with open(orig_path, 'r') as f:
    orig_dataset = json.load(f)

len(orig_dataset)

15878

In [252]:
non_ascii_samples = []

for d in orig_dataset:
    if not ' '.join(d['question']).isascii():
        non_ascii_samples.append(d)

len(non_ascii_samples)

983

#### Probing samples

In [33]:
import sdr_analysis.helpers.general_helpers
importlib.reload(sdr_analysis.helpers.general_helpers)
from sdr_analysis.helpers.general_helpers import load_pickle_list

In [227]:
_probe_task = 'link_prediction'  # link_prediction / single_node_reconstruction 
_dataset = 'spider'
_method = 'uskg'
_orig_ds = 'train'
_prob_ds = 'test'

In [228]:
X_path = f"/home/yshao/Projects/SDR-analysis/data/probing/text2sql/{_probe_task}/{_dataset}/{_method}/{_orig_ds}.{_prob_ds}.X.pkl"

# with open(X_path, 'rb') as f:
X = load_pickle_list(X_path)

/home/yshao/Projects/SDR-analysis/data/probing/text2sql/link_prediction/spider/uskg/train.test.X.pkl has only 1 object, type is <class 'list'>; return as-is


In [229]:
type(X), len(X)

(list, 14660)

In [230]:
type(X[0]), X[0].shape

(numpy.ndarray, (3072,))

In [231]:
y_path = f"/home/yshao/Projects/SDR-analysis/data/probing/text2sql/{_probe_task}/{_dataset}/{_method}/{_orig_ds}.{_prob_ds}.y.pkl"

# with open(y_path, 'r') as f:
#     y = f.read().strip().split('\n')
y = load_pickle_list(y_path)
    

/home/yshao/Projects/SDR-analysis/data/probing/text2sql/link_prediction/spider/uskg/train.test.y.pkl has only 1 object, type is <class 'list'>; return as-is


In [232]:
len(y)

14660

In [233]:
y[:10]

[2, 3, 4, 5, 6, 1, 0, 45, 49, 7]

In [234]:
Counter(y)

Counter({2: 496,
         3: 496,
         4: 496,
         5: 496,
         6: 496,
         1: 496,
         0: 496,
         45: 95,
         49: 110,
         7: 496,
         14: 496,
         8: 496,
         21: 496,
         46: 95,
         11: 496,
         10: 438,
         19: 473,
         17: 496,
         20: 496,
         50: 110,
         9: 438,
         22: 496,
         26: 496,
         24: 473,
         25: 496,
         23: 496,
         34: 496,
         28: 432,
         30: 423,
         29: 423,
         41: 287,
         47: 27,
         48: 27,
         42: 287,
         39: 197,
         43: 73,
         37: 266,
         38: 266,
         40: 197,
         44: 73,
         18: 13,
         27: 13})

In [ ]:
Counter([_y.split(' ')[0] for _y in y])

In [200]:
pos_path = f"/home/yshao/Projects/SDR-analysis/data/probing/text2sql/{_probe_task}/{_dataset}/{_method}/{_orig_ds}.{_prob_ds}.pos.txt"

with open(pos_path, 'r') as f:
    pos_lines = f.read().strip().split('\n')


In [201]:
len(pos_lines)

87206

In [202]:
pos_lines[:30], pos_lines[-30:]

(['3756\t0',
  '3756\t1',
  '3756\t2',
  '3756\t3',
  '3756\t4',
  '3756\t5',
  '3756\t6',
  '3756\t7',
  '3756\t8',
  '3756\t9',
  '3756\t10',
  '3756\t11',
  '3756\t12',
  '3756\t13',
  '3756\t14',
  '3756\t15',
  '3756\t16',
  '3756\t17',
  '3756\t18',
  '3756\t19',
  '3756\t20',
  '3756\t21',
  '3756\t22',
  '3756\t23',
  '3756\t24',
  '3756\t25',
  '3756\t26',
  '3756\t27',
  '3756\t28',
  '3756\t29'],
 ['1138\t2',
  '1138\t4',
  '1138\t5',
  '1138\t6',
  '1138\t7',
  '1138\t8',
  '1138\t9',
  '1138\t10',
  '1138\t11',
  '5971\t0',
  '5971\t1',
  '5971\t2',
  '5971\t3',
  '5971\t4',
  '5971\t6',
  '5971\t7',
  '5971\t8',
  '5971\t9',
  '5971\t10',
  '5971\t11',
  '8211\t0',
  '8211\t1',
  '8211\t2',
  '8211\t3',
  '8211\t5',
  '8211\t6',
  '8211\t7',
  '8211\t8',
  '8211\t9',
  '8211\t10'])

In [203]:
len(set([pos.split('\t')[0] for pos in pos_lines]))

4210

In [239]:
# compare data

In [235]:
_probe_task = 'link_prediction'  # link_prediction / single_node_reconstruction 
_orig_ds = 'train'
_prob_ds = 'test'

In [236]:
y1_path = f"/home/yshao/Projects/SDR-analysis/data/probing/text2sql/{_probe_task}/spider/uskg/{_orig_ds}.{_prob_ds}.y.pkl"
y1 = load_pickle_list(y1_path)
len(y1)

/home/yshao/Projects/SDR-analysis/data/probing/text2sql/link_prediction/spider/uskg/train.test.y.pkl has only 1 object, type is <class 'list'>; return as-is


14660

In [237]:
y2_path = f"/home/yshao/Projects/SDR-analysis/data/probing/text2sql/{_probe_task}/spider/lgesql/{_orig_ds}.{_prob_ds}.y.pkl"
y2 = load_pickle_list(y2_path)
len(y2)

/home/yshao/Projects/SDR-analysis/data/probing/text2sql/link_prediction/spider/lgesql/train.test.y.pkl has 14660 objects, type is <class 'numpy.int64'>; combined to a list


14660

In [238]:
y1 == y2

True

#### Tokenized len counter

In [24]:
y_tokenized = [tokenizer_fast.tokenize(_y) for _y in tqdm(y)]
len(y_tokenized)

  0%|          | 0/581931 [00:00<?, ?it/s]

581931

In [25]:
y_tokenized[:10]

[['▁question', '▁what'],
 ['▁question', '▁be'],
 ['▁question', '▁the'],
 ['▁question', '▁', "'"],
 ['▁question', '▁', '-', 'l', 'c', 'b', '-'],
 ['▁question', '▁', '\\'],
 ['▁question', '▁math', 'r', 'm'],
 ['▁question', '▁', '-', 'l', 'c', 'b', '-'],
 ['▁question', '▁', 'h'],
 ['▁question', '▁', ',']]

In [26]:
len_counter = Counter([len(_y) for _y in y_tokenized])

In [27]:
sorted(len_counter.items())

[(1, 14),
 (2, 380975),
 (3, 113910),
 (4, 36631),
 (5, 18764),
 (6, 11131),
 (7, 12137),
 (8, 3089),
 (9, 1585),
 (10, 1039),
 (11, 629),
 (12, 423),
 (13, 326),
 (14, 313),
 (15, 255),
 (16, 122),
 (17, 117),
 (18, 129),
 (19, 69),
 (20, 40),
 (21, 42),
 (22, 25),
 (23, 39),
 (24, 15),
 (25, 4),
 (26, 55),
 (27, 5),
 (29, 9),
 (30, 3),
 (32, 3),
 (33, 8),
 (35, 2),
 (36, 3),
 (38, 1),
 (44, 2),
 (51, 3),
 (72, 3),
 (92, 6),
 (134, 3),
 (182, 2)]

### T5 decoder loading
- TODO: use these in allennlp SNR model

In [53]:
from transformers import T5ForConditionalGeneration

In [54]:
t5_model = T5ForConditionalGeneration.from_pretrained('t5-large')

In [55]:
t5_model.config.decoder_start_token_id

0

In [56]:
t5_model.config.eos_token_id

1

In [74]:
t5_model.config.num_decoder_layers

24

In [58]:
t5_model.lm_head.weight

Parameter containing:
tensor([[  0.0364,  -0.2363,  -0.2236,  ...,  -0.2812,  -3.8281,  -5.9688],
        [  5.2188,  -7.0312,   7.3438,  ...,   2.7500,  14.9375,  25.1250],
        [  3.4844,   5.9688,  12.5000,  ...,  15.9375,  -4.2188,   6.4688],
        ...,
        [-20.0000,  -8.8125,   7.2188,  ...,  -5.8438,  11.0625,  23.8750],
        [-20.8750,  -7.0312,   8.5625,  ...,  -4.6562,  11.8750,  23.0000],
        [-20.6250,  -7.0000,   9.3125,  ...,  -6.5000,  11.2500,  22.0000]],
       requires_grad=True)

In [59]:
p = 55
X[p], y[p]

(array([[ 0.12346454,  0.16762893, -0.14894915, ...,  0.01294581,
         -0.12181459, -0.11360805]], dtype=float32),
 'question c')

In [60]:
tokenizer_fast.tokenize(y[p])

['▁question', '▁', 'c']

In [61]:
y_tokenized = tokenizer_fast.batch_encode_plus([y[p]], return_tensors='pt')
y_tokenized

{'input_ids': tensor([[822,   3,  75,   1]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [62]:
tokenizer_fast.convert_ids_to_tokens(y_tokenized['input_ids'][0])

['▁question', '▁', 'c', '</s>']

In [63]:
labels = y_tokenized['input_ids']
dec_input_ids = t5_model._shift_right(labels)
dec_input_ids

tensor([[  0, 822,   3,  75]])

In [64]:
enc_hidden_states = torch.tensor([X[p]])

In [65]:
dec_output = t5_model.decoder.forward(
    input_ids = dec_input_ids,
    attention_mask = y_tokenized['attention_mask'],
    encoder_hidden_states = enc_hidden_states,
)

In [66]:
dec_output.__dict__.keys()

dict_keys(['last_hidden_state', 'past_key_values', 'hidden_states', 'attentions', 'cross_attentions'])

In [67]:
dec_output.last_hidden_state.size()

torch.Size([1, 4, 1024])

In [68]:
## copied from t5 source

sequence_output = dec_output[0]
# Rescale output before projecting on vocab
# See https://github.com/tensorflow/mesh/blob/fa19d69eafc9a482aff0b59ddd96b025c0cb207d/mesh_tensorflow/transformer/transformer.py#L586
sequence_output = sequence_output * (t5_model.model_dim ** -0.5)
lm_logits = t5_model.lm_head(sequence_output)


In [69]:
lm_logits.size()

torch.Size([1, 4, 32128])

In [70]:
lm_preds = lm_logits.argmax(dim=-1)
lm_preds

tensor([[205,  75,  75,  75]])

In [71]:
_gen_toks = tokenizer_fast.convert_ids_to_tokens(lm_preds[0])
_gen_toks

['▁C', 'c', 'c', 'c']

In [73]:
tokenizer_fast.decode(lm_preds[0])

'Cccc'

### others

In [53]:
args.bert.location

't5-base'

In [62]:
args.model.__dict__

{'__self__': None,
 '__default__': {'__call__',
  '__class__',
  '__default__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__getattribute__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__iter__',
  '__le__',
  '__len__',
  '__lt__',
  '__module__',
  '__ne__',
  '__new__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__self__',
  '__setattr__',
  '__sizeof__',
  '__str__',
  '__subclasshook__',
  '__weakref__'},
 'name': 'unified.prefixtuning',
 'use_description': False,
 'concatenate_description': False,
 'map_description': False,
 'knowledge_usage': 'concatenate',
 'freeze_plm': True,
 'freeze_prefix': False}

In [51]:
print(True or 2)

True


In [52]:
print([] or 2)

2


In [53]:
txt_list = ['quick brown fox', 'old lazy dog', 'you jump I jump']

tokenized_txt_list = tokenizer_fast(txt_list, max_length=16, padding="max_length", truncation=True)

In [63]:
type(tokenized_txt_list)

transformers.tokenization_utils_base.BatchEncoding

In [65]:
tokenized_txt_list.data, tokenized_txt_list.encodings

({'input_ids': [[1704, 4216, 3, 20400, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [625, 19743, 1782, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [25, 4418, 27, 4418, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
  'attention_mask': [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]},
 [Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])])

In [146]:
print("What date was the Izmir Cup in which \u0130lhan played against Somdev Devvarman?".lower().strip().split())

['what', 'date', 'was', 'the', 'izmir', 'cup', 'in', 'which', 'i̇lhan', 'played', 'against', 'somdev', 'devvarman?']


In [157]:
"I\u0307", "\u0130", "I\u0307" == "\u0130"

('İ', 'İ', False)

In [158]:
"I\u0307".lower(), "\u0130".lower(), "I\u0307".lower() == "\u0130".lower()

('i̇', 'i̇', True)

In [159]:
len("I\u0307"), len("\u0130")

(2, 1)

In [160]:
len("I\u0307".lower()), len("\u0130".lower())

(2, 2)

In [312]:
from setuptools import setup, find_packages

In [311]:
find_packages(include=['models*', 'utils*'])

['models',
 'utils',
 'models.prompt',
 'models.adapter',
 'models.unified',
 'utils.processor']